## cryptopals challenges

### 1. Convert hex to base64

In [1]:
import base64,binascii

s = '49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d'
h = binascii.unhexlify(s)
print('hex decoded: %s'%h.decode())
b = base64.b64encode(h)
print('b64 encoded: %s'%b.decode())

hex decoded: I'm killing your brain like a poisonous mushroom
b64 encoded: SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t


### 2. Fixed XOR

In [2]:
def xor(a,b): return b''.join([bytes([a[i]^b[i]]) for i in range(len(a))])

a = binascii.unhexlify('1c0111001f010100061a024b53535009181c')
b = binascii.unhexlify('686974207468652062756c6c277320657965')
x = xor(a,b)
print('xor result: %s'%x.decode())
print('in hex: %s'%binascii.hexlify(x).decode())

xor result: the kid don't play
in hex: 746865206b696420646f6e277420706c6179


### 3. Single-byte XOR cipher

In [3]:
c = '1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736'

r = list(range(65,122))+[32]
def score(b): return sum([c in r for c in b])/len(b)   
def one_char_xor(b,x): return b''.join([bytes([c^x]) for c in b])
def brute_char(b):
    e = 0
    k = ''
    for i in range(256):
        s = score(one_char_xor(b,i))
        if s>e: e,k = s,i
    return bytes([k]),one_char_xor(b,k),e
    
b = binascii.unhexlify(c)
k,p,e = brute_char(b)
print('best possible key: %s'%k)
print('plaintext: %s'%p.decode())

best possible key: b'X'
plaintext: Cooking MC's like a pound of bacon


### 4. Detect single-character XOR

In [4]:
f = open('files/4.txt','r').read().splitlines()

s,n,y,a = 0,0,'',''

for i,l in enumerate(f):
    k,p,e = brute_char(binascii.unhexlify(l))
    if e > s: s,n,y,a = e,i,k,p
    
print('best possible line: %s'%n)
print('best possible key: %s'%y)
print('plaintext: %s'%a.decode())

best possible line: 170
best possible key: b'5'
plaintext: Now that the party is jumping




### 5. Implement repeating-key XOR

In [5]:
b = b'''Burning 'em, if you ain't quick and nimble
I go crazy when I hear a cymbal'''
k = b'ICE'

def xor(b,k): return b''.join([bytes([b[i]^k[i%len(k)]]) for i in range(len(b))])

print('result of keyed xor: %s'%binascii.hexlify(xor(p,k)).decode())

result of keyed xor: 3b072a4f40182547657037e8110f1521106f562823002d3e254d2c395b5f


### 6. Break repeating-key XOR

In [6]:
from itertools import combinations

def hamming(a,b): return sum([sum(int(j) for j in bin(a[i]^b[i])[2:]) for i in range(len(a))])

a = b'this is a test'
b = b'wokka wokka!!!'
print('hamming distance between test strings: %s'%hamming(a,b))

def find_key_size(c):
    kn = 0
    d = 999999
    
    for i in range(2,40,1):
        p = combinations([c[j*i:(j+1)*i] for j in range(5)],2)
        t = 0
        for (x,y) in p: t += hamming(x,y)/i
        if (t < d): kn,d = i,t
    return kn

def break_vigenere(c):
    kn = find_key_size(c)
    print("key size: %s"%kn)
    A = [[] for _ in range(kn)]
    for i in range(len(c)): A[i%kn].append(c[i])

    y = b''
    for i in range(kn): y += brute_char(A[i])[0]
    return y,xor(c,y)
        
        
#     key = ''.join(bests)
#     print("[+] found best possible key:",key)
#     return keyed_xor(ciphertext.encode('utf-8'),key.encode('utf-8')).decode()

c = base64.b64decode(open('files/6.txt','r').read())
k,p = break_vigenere(c)
print('key: %s'%k.decode())
print('decrypted plaintext:')
print(p.decode())

hamming distance between test strings: 37
key size: 29
key: Terminator X: Bring the noise
decrypted plaintext:
I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Su

### 7. AES in ECB mode

In [7]:
from Crypto.Cipher import AES

def AES_ECB_decrypt(ciphertext,key):
    aes = AES.new(key, AES.MODE_ECB)
    return aes.decrypt(ciphertext)

def AES_ECB_encrypt(plaintext,key):
    aes = AES.new(key, AES.MODE_ECB)
    return aes.encrypt(plaintext)

c = base64.b64decode(open('files/7.txt','r').read())
k = b"YELLOW SUBMARINE"
a = AES.new(k,AES.MODE_ECB)
p = a.decrypt(c).decode()
print('plaintext:')
print(p)

plaintext:
I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make

### 8. Detect AES in ECB mode

In [8]:
def chunkit(s,n): return [s[i:i+n] for i in range(0,len(s),n)]

f = open('files/8.txt','r').read().splitlines()
for i,l in enumerate(f):
    c = chunkit(l,AES.block_size)
    r = len(c) - len(set(c))
    if r: print('possible ECB at line %s'%i)

possible ECB at line 132


### 9. Implement PKCS#7 padding

In [9]:
def pad(s,n):
    i = -len(s)%n
    if not i: i = n
    return s + bytes([i])*i

s = b'YELLOW SUBMARINE'
print('padded string:',pad(s,20))

padded string: b'YELLOW SUBMARINE\x04\x04\x04\x04'


### 10. Implement CBC mode

In [10]:
def unpad(s):
    c = s[-1]
    if not sum([d!=c for d in s[-c:]]): return s[:-c]
    else: return False

def decrypt(c,i,k):
    cs = [i] + chunkit(c,AES.block_size)
    a = AES.new(k,AES.MODE_ECB)
    return b''.join([xor(cs[i],a.decrypt(cs[i+1])) for i in range(len(cs)-1)])
    
c = base64.b64decode(open('files/10.txt','r').read())
i = b'\x00'*16
k = b'YELLOW SUBMARINE'
p = unpad(decrypt(c,i,k)).decode()
print('decrypted plaintext:')
print(p)

decrypted plaintext:
I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly gir

### 11. An ECB/CBC detection oracle

In [11]:
import random
from Crypto import Random

def keygen(n): return Random.new().read(n)

def random_encryptor(p):
    p = keygen(random.randint(5,10)) + p + keygen(random.randint(5,10))
    p = pad(p,AES.block_size)                      
    t = random.randint(0,1)
    a = AES.new(keygen(AES.block_size),AES.MODE_ECB if t else AES.MODE_CBC)
    print('random encryptor using %s'%('ECB' if t else 'CBC'))
    return a.encrypt(p)

def ECB_CBC_oracle(encryptor):
    c = encryptor(b'lol'*32)
    cs = chunkit(c,AES.block_size)
    return 'ECB' if len(cs)-len(set(cs)) else 'CBC'

print('oracle detected %s'%ECB_CBC_oracle(random_encryptor))

random encryptor using CBC
oracle detected CBC


### 12. Byte-at-a-time ECB decryption (Simple)

In [12]:
key = key_gen(AES.block_size)
unknown_string = b64decode('Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkgaGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBqdXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUgYnkK')

def consistent_encryptor(plaintext): return AES_ECB_encrypt(add_pad(plaintext.encode('utf-8') + unknown_string, AES.block_size), key)

def find_block_size(encryptor):
    teststr = ''
    base_length = len(encryptor(teststr))
    new_length = base_length
    while new_length == base_length: 
        teststr += 'A'
        new_length = len(encryptor(teststr))
    return new_length - base_length

def find_next_byte(encryptor,known_part,block_size,trash_bytes=0):
    comparison_length = block_size*((len(known_part)+trash_bytes)//block_size+1)
    front_part = 'A'*(comparison_length-trash_bytes-len(known_part)-1)
    target_string = encryptor(front_part)[:comparison_length]
    for i in range(256):
        if encryptor(front_part + known_part + chr(i))[:comparison_length] == target_string: return chr(i)
    return ''
    
def ECB_decoder(encryptor):
    print('[=] byte-at-a-time attack on ECB')
    block_size = find_block_size(encryptor)
    print('[+] block size found:',block_size)
    print('[=] oracle detected', oracle(encryptor))
    found = ''
    for n in range(len(encryptor(''))): found += find_next_byte(encryptor,found,block_size)
    return found
    
plaintext = ECB_decoder(consistent_encryptor)
print('\n[+] DECRYPTED:\n' + plaintext)


[=] byte-at-a-time attack on ECB
[+] block size found: 16
[=] oracle detected ECB

[+] DECRYPTED:
Rollin' in my 5.0
With my rag-top down so my hair can blow
The girlies on standby waving just to say hi
Did you stop? No, I just drove by



### 13. ECB cut-and-paste

In [13]:
key = key_gen(AES.block_size)

def string_parser(string): return {pair.split('=')[0]:pair.split('=')[1] for pair in string.split('&')}
def dict_parser(dict): return "&".join([key+"="+dict[key] for key in dict.keys()])
def profile_for(email): return {'email':email.replace('&','').replace('=',''),'uid':'10','role':'user'}
def encrypt_profile(email): return AES_ECB_encrypt(add_pad(dict_parser(profile_for(email)).encode('utf-8'), AES.block_size), key)
def decrypt_profile(bytestring): return string_parser(AES_ECB_decrypt(bytestring, key).decode().replace('\x00',''))

def profile_hacker(encryption,email):
    print('[=] performing injection on profile')
    block_size = find_block_size(encryption)
    offset = -(len(email) + len('email=&uid=10&role='))%block_size
    email = "X"*offset + email
    print('[=] padded email:', email)
    admin_ciphertext = encryption('x'*(block_size - len('email:')) + 'admin' + '\x00'*(block_size-5))[block_size:2*block_size]
    front_ciphertext = encryption(email)[:len(email)+len('email=&uid=10&role=')]
    return front_ciphertext+admin_ciphertext, email
    
email = 'haha@lol.com'
profile_string, new_mail = profile_hacker(encrypt_profile,email)
print('[+] encrypted profile generated:', profile_string)
print('[+] decryped profile:')
print(decrypt_profile(profile_string))

[=] performing injection on profile
[=] padded email: Xhaha@lol.com
[+] encrypted profile generated: b'\xbd\xb4\xef"\xcb\x82[\x12\nU\xbb\x1fV/\xe0{\xa3\x9f\x86\x1c\x84\xd3\x8b\x81\x17\x14\xc3\x1e\x96\x91\xca\xa3Cg\x00\xf2\x96\xb3\xa8\xb3\x1fB\x00\xdb\xd8,\x99\x18'
[+] decryped profile:
{'email': 'Xhaha@lol.com', 'uid': '10', 'role': 'admin'}


### 14. Byte-at-a-time ECB decryption (Harder)

In [14]:
key = key_gen(AES.block_size)
unknown_string = b64decode('Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkgaGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBqdXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUgYnkK')
prefix_length = random.randint(0,AES.block_size*4)

def harder_encryptor(plaintext): return AES_ECB_encrypt(add_pad(b'\x00'*prefix_length + plaintext.encode('utf-8') + unknown_string, AES.block_size), key)

def count_trash_blocks(encryptor,block_size):
    c1 = encryptor('A')
    c2 = encryptor('B')
    i = 0
    while (c1[i]==c2[i]): i += 1
    return i//block_size

def count_trash_bytes(encryptor,block_size,trash_blocks):
    comparison_length = block_size*(1+trash_blocks)
    for test_offset in range(block_size):
        front_part = 'A'*test_offset
        target_string = encryptor(front_part)[:comparison_length]
        for i in range(256):
            if encryptor(front_part + chr(i))[:comparison_length] == target_string: return block_size*(trash_blocks+1)-(1+test_offset)

def harder_ECB_decoder(encryptor):
    print('[=] byte-at-a-time attack on ECB')
    block_size = find_block_size(encryptor)
    print('[+] block size detected:',block_size)
    print('[=] oracle detected', oracle(encryptor))
    trash_blocks = count_trash_blocks(encryptor,block_size)
    trash_bytes = count_trash_bytes(encryptor,block_size,trash_blocks)
    print('[+] prefix length found', trash_bytes)
    found = ''
    for n in range(len(encryptor(''))): found += find_next_byte(encryptor,found,block_size,trash_bytes)
    print('[+] done')
    return found    

plaintext = harder_ECB_decoder(harder_encryptor)
print('\nDECRYPTED:\n' + plaintext)


[=] byte-at-a-time attack on ECB
[+] block size detected: 16
[=] oracle detected ECB
[+] prefix length found 11
[+] done

DECRYPTED:
Rollin' in my 5.0
With my rag-top down so my hair can blow
The girlies on standby waving just to say hi
Did you stop? No, I just drove by



### 15. PKCS#7 padding validation

In [15]:
str1 = "ICE ICE BABY\x04\x04\x04\x04"
print('bytestring:',str1.encode())
print('padding validation:',remove_padding(str1)[0])
print('unpadded:',remove_padding(str1)[1])
str2 = "ICE ICE BABY\x01\x02\x03\x04"
print('\nbytestring:',str2.encode())
print('padding validation:',remove_padding(str2)[0])
print('unpadded:',remove_padding(str2)[1])

bytestring: b'ICE ICE BABY\x04\x04\x04\x04'
padding validation: True
unpadded: b'ICE ICE BABY'

bytestring: b'ICE ICE BABY\x01\x02\x03\x04'
padding validation: False
unpadded: bad padding


### 16. CBC bitflipping attacks

In [16]:
key = key_gen(AES.block_size)
iv = key_gen(AES.block_size)

def encrypt_cookie_CBC(plaintext): 
    string = "comment1=cooking%20MCs;userdata=" + plaintext.replace(';','').replace('=','') + ";comment2=%20like%20a%20pound%20of%20bacon"
    return CBC_encrypt(add_pad(string,AES.block_size),iv,key)

def find_admin(ciphertext):
    plaintext = CBC_decrypt(ciphertext,iv,key)
    parsed = {pair.split(b"=")[0]:pair.split(b"=")[1] for pair in plaintext.split(b";")}
    print(parsed)
    if b'admin' in parsed.keys() and parsed[b'admin']==b'true': return True
    else: return False

def bitflipper(encryptor):
    print('[=] perfroming bit flipping attack')
    ciphertext = encryptor('AadminBtrue')
    plaintext = CBC_decrypt(ciphertext,iv,key).decode()
    A_index = plaintext.find("Aadmin") - AES.block_size
    B_index = plaintext.find("Btrue") - AES.block_size
    replace_A = ciphertext[A_index] ^ ord("A") ^ ord(";")
    replace_B = ciphertext[B_index] ^ ord("B") ^ ord("=")
    return ciphertext[:A_index] + bytes([replace_A]) + ciphertext[A_index+1:B_index] + bytes([replace_B]) + ciphertext[B_index+1:]
       
modified_cookie = bitflipper(encrypt_cookie_CBC)
print("[+] done! modified cookie:", modified_cookie)
print("[+] decrypted cookie: ",end="")
print('[+] oracle verification:',find_admin(modified_cookie))

[=] perfroming bit flipping attack
[+] done! modified cookie: b'9\x8f\xac\xc4\x1c\xa6~\xa5\x89\x17Rs\xb0\x19k\xda\x92M\xca\xa0\x04:\xed\xc7\xa3\x8aW\xc2\xff\xc0e\xe2\x8dT\'lIK~\xc0^}\xf1)0\xec\xaaf\x87\x03mV\xed\r\xd7\xe7\x80\x11\xd3.<\x99 \xf2\xbc\xa0\xfa\x92iP\x18["nu?\xd2z\xa0\x872\xdf\xa6mt\xeb~q\xdaw\x12t\xb8\xb3?\xe6\x13\x81W\xb2`o\x9f\xa2n\x9e\x18\x83\xee\x95\x89A'
[+] decrypted cookie: {b'comment1': b'cooking\xfe\x82\xdc[_\xc3u\xb7\x00\xfa\xae\x1f\xcf\x83\xeaF', b'admin': b'true', b'comment2': b'%20like%20a%20pound%20of%20bacon\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10'}
[+] oracle verification: True


### 17. The CBC padding oracle

In [18]:
key = key_gen(AES.block_size)
    
def encrypt_random_line(file):
    lines = open(file,'r').readlines()
    plaintext = b64decode(lines[random.randint(0,9)].strip())
    iv = key_gen(AES.block_size)
    return CBC_encrypt(plaintext,iv,key),iv

def padding_oracle(ciphertext,iv): return remove_padding(CBC_decrypt(ciphertext,iv,key))[0]

def attack_one_byte(target_block,iv,oracle,guessed_intermediate,n):
    end_part = keyed_xor(guessed_intermediate,bytes([n+1])*n)
    candidates = []
    for i in range(256):
        test_iv = iv[:-n-1] + bytes([i]) + end_part
        try:
            if oracle(target_block,test_iv): candidates.append(i)
        except: continue
    return candidates
    
def attack_one_block(target_block,iv,oracle):
    guessed_intermediate = b''
    for n in range(len(iv)):
        candidates = attack_one_byte(target_block,iv,oracle,guessed_intermediate,n)
        if len(candidates)==1: guessed_intermediate = bytes([candidates[0]^(n+1)]) + guessed_intermediate
        else: 
            for candidate in candidates:
                trial_intermediate = bytes([candidate^(n+1)]) + guessed_intermediate
                next_candidates = attack_one_byte(target_block,iv,oracle,trial_intermediate,n+1)
                if next_candidates: guessed_intermediate = bytes([candidate^(n+1)]) + guessed_intermediate
    return keyed_xor(guessed_intermediate,iv)

def attack_padding_oracle(ciphertext,iv,oracle):
    print('[=] attacking padding oracle')
    chunks = [iv] + [ciphertext[n:n+AES.block_size] for n in range(0,len(ciphertext),AES.block_size)]
    return b''.join([attack_one_block(chunks[i+1],chunks[i],oracle) for i in range(len(chunks)-1)])
    
ciphertext,iv = encrypt_random_line('files/17.txt')
plaintext = remove_padding(attack_padding_oracle(ciphertext,iv,padding_oracle))[1].decode()
print("[+] done! decrypted plaintext:",plaintext)

[=] attacking padding oracle
[+] done! decrypted plaintext: 000002Quick to the point, to the point, no faking


### 18. Implement CTR, the stream cipher mode

In [19]:
import struct

def keymaker(key,nonce,ctr): return AES_ECB_encrypt(struct.pack('<QQ', nonce, ctr),key)
    
def CTR_decrypt(ciphertext,key,nonce):
    ctr = 0
    chunks = [ciphertext[n:n+len(key)] for n in range(0,len(ciphertext),len(key))]
    plaintext = b""
    for chunk in chunks:
        plaintext += keyed_xor(chunk,keymaker(key,nonce,ctr))
        ctr += 1
    return plaintext


def CTR_encrypt(plaintext,key,nonce):
    ctr = 0
    chunks = [plaintext[n:n+len(key)] for n in range(0,len(plaintext),len(key))]
    ciphertext = b""
    for chunk in chunks:
        ciphertext += keyed_xor(chunk,keymaker(key,nonce,ctr))
        ctr += 1
    return ciphertext

ciphertext = b64decode('L77na/nrFsKvynd6HzOoG7GHTLXsTVu9qvY/2syLXzhPweyyMTJULu/6/kXX0KSvoOLSFQ==')
print('plaintext:',CTR_decrypt(ciphertext,b"YELLOW SUBMARINE",0).decode())


plaintext: Yo, VIP Let's kick it Ice, Ice, baby Ice, Ice, baby 


### 19. Break fixed-nonce CTR mode using substitutions

In [20]:
def fixed_nonce_encryption(file,key,nonce):
    plaintexts = [b64decode(line.strip()) for line in open(file,'r').readlines()]
    ciphertexts = [CTR_encrypt(plaintext,key,nonce) for plaintext in plaintexts]
    return ciphertexts,plaintexts

def attack_fixed_nonce_encryption(ciphertexts):
    print('[=] attacking fixed nonce encryption')
    longest_length = max([len(ciphertext) for ciphertext in ciphertexts])
    keystream = []
    for l in range(longest_length):
        target = [ciphertext[l] for ciphertext in ciphertexts if len(ciphertext)>l]
        keystream.append(ord(one_char_brute(target)[0]))
        if not (l+1)%20: print('[=] {} characters done'.format(l+1))
    print('[+] done!')
    return keystream
    
ciphertexts,plaintexts = fixed_nonce_encryption('files/19.txt',key_gen(AES.block_size),0)
keystream = attack_fixed_nonce_encryption(ciphertexts)
cracked_plaintexts = [keyed_xor(ciphertext,keystream) for ciphertext in ciphertexts]
print('\nCHECK:')
for real,crack in zip(plaintexts,cracked_plaintexts):
    print("actual plaintext:",real.decode())
    print("cracked plaintext:",crack.decode())
    print('\n')


[=] attacking fixed nonce encryption
[=] 20 characters done
[+] done!

CHECK:
actual plaintext: I have met them at close of day
cracked plaintext: i have met them at close of daX


actual plaintext: Coming with vivid faces
cracked plaintext: coming with vivid faces


actual plaintext: From counter or desk among grey
cracked plaintext: from counter or desk among greX


actual plaintext: Eighteenth-century houses.
cracked plaintext: eighteenth-century houses.


actual plaintext: I have passed with a nod of the head
cracked plaintext: i have passed with a nod of thD hic\


actual plaintext: Or polite meaningless words,
cracked plaintext: or polite meaningless words,


actual plaintext: Or have lingered awhile and said
cracked plaintext: or have lingered awhile and saHd


actual plaintext: Polite meaningless words,
cracked plaintext: polite meaningless words,


actual plaintext: And thought before I had done
cracked plaintext: and thought before I had done


actual plaintext: Of a mocking 

### 20. Break fixed-nonce CTR statistically

In [21]:
ciphertexts,plaintexts = fixed_nonce_encryption('files/20.txt',key_gen(AES.block_size),0)
keystream = attack_fixed_nonce_encryption(ciphertexts)
cracked_plaintexts = [keyed_xor(ciphertext,keystream) for ciphertext in ciphertexts]
print('\nCHECK:')
for real,crack in zip(plaintexts,cracked_plaintexts):
    print("actual plaintext:",real.decode())
    print("cracked plaintext:",crack.decode())
    print('\n')


[=] attacking fixed nonce encryption
[=] 20 characters done
[=] 40 characters done
[=] 60 characters done
[=] 80 characters done
[=] 100 characters done
[+] done!

CHECK:
actual plaintext: I'm rated "R"...this is a warning, ya better void / Poets are paranoid, DJ's D-stroyed
cracked plaintext: N'm rated "R"...this is a warning, ya better void / Poets are paranoid, DJ's D-stroyed


actual plaintext: Cuz I came back to attack others in spite- / Strike like lightnin', It's quite frightenin'!
cracked plaintext: Duz I came back to attack others in spite- / Strike like lightnin', It's quite frightenin'!


actual plaintext: But don't be afraid in the dark, in a park / Not a scream or a cry, or a bark, more like a spark;
cracked plaintext: Eut don't be afraid in the dark, in a park / Not a scream or a cry, or a bark, more like a sparj:


actual plaintext: Ya tremble like a alcoholic, muscles tighten up / What's that, lighten up! You see a sight but
cracked plaintext: ^a tremble like a alcoholi

actual plaintext: A pen and a paper, a stereo, a tape of / Me and Eric B, and a nice big plate of
cracked plaintext: F pen and a paper, a stereo, a tape of / Me and Eric B, and a nice big plate of


actual plaintext: Fish, which is my favorite dish / But without no money it's still a wish
cracked plaintext: Aish, which is my favorite dish / But without no money it's still a wish


actual plaintext: 'Cuz I don't like to dream about gettin' paid / So I dig into the books of the rhymes that I made
cracked plaintext:  Cuz I don't like to dream about gettin' paid / So I dig into the books of the rhymes that I maed


actual plaintext: So now to test to see if I got pull / Hit the studio, 'cuz I'm paid in full
cracked plaintext: To now to test to see if I got pull / Hit the studio, 'cuz I'm paid in full


actual plaintext: Rakim, check this out, yo / You go to your girl house and I'll go to mine
cracked plaintext: Uakim, check this out, yo / You go to your girl house and I'll go to mine


act

### 21. Implement the MT19937 Mersenne Twister RNG

In [22]:
class mersenne_rng:
    (w, n, m, r) = (32, 624, 397, 31)
    a = 0x9908B0DF
    (u, d) = (11, 0xFFFFFFFF)
    (s, b) = (7, 0x9D2C5680)
    (t, c) = (15, 0xEFC60000)
    l = 18
    lower_mask = (1 << r) - 1
    upper_mask = (not lower_mask)&(2**w-1)
    f = 1812433253
    
    def __init__(self,seed):
        self.index = self.n
        self.mt = [seed]
        for i in range(1,self.n,1): self.mt.append((self.f * (self.mt[-1] ^ (self.mt[-1] >> (self.w-2))) + i)&(2**self.w-1))
        
    def extract_number(self):
        if self.index >= self.n: self.twist()
        y = self.mt[self.index]
        y ^= ((y >> self.u) & self.d)
        y ^= ((y << self.s) & self.b)
        y ^= ((y << self.t) & self.c)
        y ^= y >> self.l
        self.index += 1
        return y&(2**self.w-1)

    def twist(self):
        for i in range(self.n):
            x = (self.mt[i] & self.upper_mask) + (self.mt[(i+1) % self.n] & self.lower_mask)
            xA = x >> 1
            if x%2: xA ^= self.a
            self.mt[i] = self.mt[(i + self.m) % self.n] ^ xA
        self.index = 0
        pass
    
rng = mersenne_rng(5489)
print('extracting numbers from rng:')
for i in range(3): print(rng.extract_number())
print('\nverification:')
rng = mersenne_rng(5489)
for i in range(3): print(rng.extract_number())

extracting numbers from rng:
3499211612
581869302
2750016492

verification:
3499211612
581869302
2750016492



### 22. Crack an MT19937 seed

In [23]:
def brute_rng(rng,brute_max,output,verbose=0):
    if verbose: print("[=] performing brute force for output",output)
    for i in range(brute_max):
        test_rng = rng(i)
        if test_rng.extract_number() == output: return i
    
seed = random.randint(40,1000)
new_time = seed + random.randint(40,1000)
rng = mersenne_rng(seed)
output = rng.extract_number()
seed_guess = brute_rng(mersenne_rng,new_time,output,1)
print("[+] done! cracked seed:", seed_guess)
print("[+] real seed:", seed)

[=] performing brute force for output 3811360942
[+] done! cracked seed: 410
[+] real seed: 410


### 23. Clone an MT19937 RNG from its output

In [24]:
def get_nth_bit(num,bit,length): 
    if (bit > 0 and bit <= length): return (num >> (bit-1))&1
    else: return 0

def set_nth_bit(num,bit): return num^int(2**(bit-1))

def undo_last_step(target,distance,length):
    undone = 0
    for i in range(length):
        bit_here = get_nth_bit(target,length-i-1,length) ^ get_nth_bit(undone,length-1-i+distance,length)
        if bit_here: undone = set_nth_bit(undone,length-i-1)
    return undone

def undo_middle_steps(target,distance,mask,length):
    undone = 0
    for i in range(length):
        bit_here = get_nth_bit(target,i+1,length) ^ (get_nth_bit(undone,i-distance+1,length) & get_nth_bit(mask,i+1,length))
        if bit_here: undone = set_nth_bit(undone,i+1)
    return undone

def undo_first_step(target,distance,mask,length):
    undone = 0
    for i in range(length):
        bit_here = get_nth_bit(target,length-i-1,length) ^ (get_nth_bit(undone,length-1-i+distance,length) & get_nth_bit(mask,length-i-1,length))
        if bit_here: undone = set_nth_bit(undone,length-i-1)
    return undone

class mersenne_cloner:
    
    def __init__(self,rng):
        self.w = rng.w
        self.l = rng.l
        self.t = rng.t
        self.c = rng.c
        self.s = rng.s
        self.b = rng.b
        self.u = rng.u
        self.d = rng.d
        self.n = rng.n
        self.rng = rng
    
    def untemper(self,output):
        output = undo_last_step(output,self.l,self.w)
        output = undo_middle_steps(output,self.t,self.c,self.w)
        output = undo_middle_steps(output,self.s,self.b,self.w)
        output = undo_first_step(output,self.u,self.d,self.w)
        return output
    
    def clone(self):
        mt = []
        for i in range(self.n): mt += [self.untemper(self.rng.extract_number())]
        new_rng = mersenne_rng(0)
        new_rng.mt = mt
        return new_rng
 
seed = random.randint(0,99999)
original_rng = mersenne_rng(seed)
cloned_rng = mersenne_cloner(original_rng).clone()

print('cloned rng predictions:')
for _ in range(10): print(cloned_rng.extract_number(),end=',')

cloned rng predictions:
277862139,649510149,2039020650,3278738437,2117132928,2604782870,3365459108,3452721146,3285071831,1222901948,

### 24. Create the MT19937 stream cipher and break it

In [25]:
def rng_encrypt(seed,plaintext):
    rng = mersenne_rng(seed)
    chunks = [plaintext[n:n+4] for n in range(0,len(plaintext),4)]
    ciphertext = b""
    
    for chunk in chunks:
        keynumber = rng.extract_number()
        keystream = [keynumber//(255**i)%255 for i in range(4)]
        ciphertext += keyed_xor(chunk,keystream)
    return ciphertext

def rng_decrypt(seed,ciphertext):
    rng = mersenne_rng(seed)
    chunks = [ciphertext[n:n+4] for n in range(0,len(ciphertext),4)]
    plaintext = b""
    
    for chunk in chunks:
        keynumber = rng.extract_number()
        keystream = [keynumber//(255**i)%255 for i in range(4)]
        plaintext += keyed_xor(chunk,keystream)
    return plaintext

def known_plaintext_attack(plaintext,ciphertext,verbose=0):
    keystream = [byte for byte in keyed_xor(plaintext,ciphertext)[:4]]
    keynumber = sum([keystream[i]*(255**i) for i in range(4)])
    return brute_rng(mersenne_rng,1000,keynumber,verbose)
    
def get_otp(random_string): 
    time = random.randint(0,1000)
    return rng_encrypt(time,random_string)

def check_otp(test_otp,random_string): return bool(known_plaintext_attack(test_otp,random_string))
    
seed = random.randint(0,1000)

print("CHECK RNG ENCRYPTION")
print("==================")
plaintext = b'random unknown characters' + b'A'*14
print('plaintext',plaintext)
ciphertext = rng_encrypt(seed,plaintext)
print('ciphertext',ciphertext)
decrypt_test = rng_decrypt(seed,ciphertext).decode()
print("decrypted:",decrypt_test)

print("\n\nBRUTE FORCE ATTACK ON SEED")
print("==================")
cracked_seed = known_plaintext_attack(plaintext,ciphertext,1)
print("[+] done! cracked seed:",cracked_seed)
print("[+] real seed:",seed)

print("\n\nRNG OTP IMPLEMENTATION")
print("==================")
random_string = key_gen(4)
otp = get_otp(random_string)
print("otp:",otp)
print("check on right otp:",check_otp(get_otp(random_string),random_string))
print("check on wrong otp:",check_otp(key_gen(4),random_string))

CHECK RNG ENCRYPTION
plaintext b'random unknown charactersAAAAAAAAAAAAAA'
ciphertext b"px\xba\xd8R\xb8$\x04\xc6\x1a1\xfe\xcd\x18\xfdng\xa6y<\xc2\x145\x1d\xbd!\xba'\xe2\xff\x99'hr!\x8b\x1c\xd3\x07"
decrypted: random unknown charactersAAAAAAAAAAAAAA


BRUTE FORCE ATTACK ON SEED
[=] performing brute force for output 3131090177
[+] done! cracked seed: 28
[+] real seed: 28


RNG OTP IMPLEMENTATION
otp: b'{j\xa6h'
check on right otp: True
check on wrong otp: False



### 25. Break "random access read/write" AES CTR

In [26]:
def edit(ciphertext,offset,newtext,nonce):
    plaintext = CTR_decrypt(ciphertext,key,nonce)
    new_plaintext = plaintext[:offset] + newtext + plaintext[offset+len(newtext):]
    return CTR_encrypt(new_plaintext,key,nonce)
    
def attack_edit_function(edit_function,ciphertext,nonce):
    print('[=] attacking edit function')
    chosen_plaintext = b'\x00'*len(ciphertext)
    new_ciphertext = edit_function(ciphertext,0,chosen_plaintext,nonce)
    return keyed_xor(new_ciphertext,ciphertext)
    
    
plaintext = b64decode(open('files/25.txt','r').read())
key = key_gen(16)
ciphertext = CTR_encrypt(plaintext,key,0)
cracked_plaintext = attack_edit_function(edit,ciphertext,0)
print('[+] done')
print('[+] checking plaintext:',cracked_plaintext == plaintext)

[=] attacking edit function
[+] done
[+] checking plaintext: True


### 26. CTR bitflipping

In [27]:
key = key_gen(AES.block_size)
nonce = 0

def encrypt_cookie_CTR(plaintext): 
    string = "comment1=cooking%20MCs;userdata=" + plaintext.replace(';','').replace('=','') + ";comment2=%20like%20a%20pound%20of%20bacon"
    return CTR_encrypt(string.encode(),key,nonce)

def find_admin_CTR(ciphertext):
    plaintext = CTR_decrypt(ciphertext,key,nonce)
    parsed = {pair.split(b"=")[0]:pair.split(b"=")[1] for pair in plaintext.split(b";")}
    print(parsed)
    if b'admin' in parsed.keys() and parsed[b'admin']==b'true': return True
    else: return False

def bitflipper_CTR(encryptor):
    print('[=] bitflipping attack on CTR')
    ciphertext = encryptor('AadminBtrue')
    plaintext = CTR_decrypt(ciphertext,key,nonce).decode()
    A_index = plaintext.find("Aadmin")
    B_index = plaintext.find("Btrue")
    replace_A = ciphertext[A_index] ^ ord("A") ^ ord(";")
    replace_B = ciphertext[B_index] ^ ord("B") ^ ord("=")
    return ciphertext[:A_index] + bytes([replace_A]) + ciphertext[A_index+1:B_index] + bytes([replace_B]) + ciphertext[B_index+1:]

modified_cookie = bitflipper_CTR(encrypt_cookie_CTR)
print("[+] done! modified cookie:", modified_cookie)
print("\nCHECK:")
print(find_admin_CTR(modified_cookie))

[=] bitflipping attack on CTR
[+] done! modified cookie: b'\x95\x95\x01\x13p\xe3\x13\x9eE\x8a\x0f<\x07\xdb\n\xa5}a\xf5J\\\xd5\x0b\x03\xbd{\x94\xac?\xef\xd9j.\xbb\x061\x82\xd1\xce\x1d\xeb\xe8E-\xac\xd5\xff\xb01L\xc5\xf9u\xc4\xf5K\x92\x1bC\x9e\xde\x0b\xb99&\x1c\xad\x07\xbcv\x9a\xecN\xb9\x05%\x9e\xd5\x1a\x96O-\xe7\x8f\xd6'

CHECK:
{b'comment1': b'cooking%20MCs', b'userdata': b'', b'admin': b'true', b'comment2': b'%20like%20a%20pound%20of%20bacon'}
True


### 27. Recover the key from CBC with IV=Key

In [28]:
key = key_gen(AES.block_size)
plaintext = '''I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy.'''.replace('\n','')
ciphertext = CBC_encrypt(plaintext,key,key)

def CBC_decrypt_and_check(ciphertext):
    possible_plaintext = remove_padding(CBC_decrypt(ciphertext,key,key))[1] if remove_padding(CBC_decrypt(ciphertext,key,key))[0] else CBC_decrypt(ciphertext,key,key)
    compliant = sum([char>=32 and char<=122 for char in possible_plaintext])==len(possible_plaintext)
    if compliant: return True,possible_plaintext
    else: return False,possible_plaintext

def IV_is_key_attack(ciphertext):
    print('[=] attacking IV=key ciphertext')
    C1 = ciphertext[:AES.block_size]
    modified_ciphertext = C1 + b'\x00'*AES.block_size + C1
    error_plaintext = CBC_decrypt_and_check(modified_ciphertext)[1]
    P1 = error_plaintext[:AES.block_size]
    P3 = error_plaintext[AES.block_size*2:AES.block_size*3]
    return keyed_xor(P1,P3)
    
cracked_key = IV_is_key_attack(ciphertext)
print("[+] done! cracked key:",cracked_key)
print("[+] real key:",key)

[=] attacking IV=key ciphertext
[+] done! cracked key: b'\xea\xdf\xbe\xad\xf5DK\x12"\x9b\xdb\x94\xf6\x9eNR'
[+] real key: b'\xea\xdf\xbe\xad\xf5DK\x12"\x9b\xdb\x94\xf6\x9eNR'


### 28. Implement a SHA-1 keyed MAC

In [29]:
from hashlib import sha1
key = key_gen(40)

def leftrotate(val,distance,limit): return ((val << distance) % (1 << limit)) | (val >> (limit - distance))

def my_sha1(message,ml=0,h0=0x67452301,h1=0xEFCDAB89,h2=0x98BADCFE,h3=0x10325476,h4=0xC3D2E1F0):
    if isinstance(message,str): message = message.encode()
    if not ml: ml = len(message)*8
    message += b'\x80'
    message += b'\x00'*((56-len(message))%64)
    message += struct.pack('>Q',ml)
    chunks = [message[n:n+64] for n in range(0,len(message),64)]
    for chunk in chunks:
        words = [struct.unpack('>I',chunk[n:n+4])[0] for n in range(0,len(chunk),4)]
        for i in range(16,80): words.append(leftrotate(words[i-3] ^ words[i-8] ^ words[i-14] ^ words[i-16],1,32))
        (a,b,c,d,e) = (h0,h1,h2,h3,h4)
        
        for i in range(80):
            if i in list(range(0,20)):           
                f = d ^ (b & (c ^ d))
                k = 0x5A827999
            elif i in list(range(20,40)):
                f = b ^ c ^ d
                k = 0x6ED9EBA1
            elif i in list(range(40,60)):
                f = (b & c) | (d & (b | c))
                k = 0x8F1BBCDC
            else:
                f = b ^ c ^ d
                k = 0xCA62C1D6

            temp = leftrotate(a,5,32) + f + e + k + words[i] & 0xffffffff
            e = d
            d = c
            c = leftrotate(b,30,32)
            b = a
            a = temp

        h0 = (h0 + a) & 0xffffffff
        h1 = (h1 + b) & 0xffffffff
        h2 = (h2 + c) & 0xffffffff
        h3 = (h3 + d) & 0xffffffff
        h4 = (h4 + e) & 0xffffffff
    
    return '%08x%08x%08x%08x%08x'%(h0,h1,h2,h3,h4)
    

def sha1_mac(message,key): return my_sha1(key+message)

test_message = b'message to test'
print('message:',test_message.decode())
print('sha1 hash:', my_sha1(test_message))
print('checking sha1 implementation:', my_sha1(test_message) == sha1(test_message).hexdigest())

mac = sha1_mac(test_message,key)
print("\nMAC:",mac)
print("authenticate right message: ", mac == sha1_mac(test_message,key))
print("authenticate wrong message: ", mac == sha1_mac(key_gen(len(test_message)),key))

message: message to test
sha1 hash: 92516ad7301718d67433635ef746404c4b6aebc9
checking sha1 implementation: True

MAC: d5a29f05354904d76ff92058f067095b87210f4c
authenticate right message:  True
authenticate wrong message:  False


### 29. Break a SHA-1 keyed MAC using length extension

In [30]:
def get_mac_padding(message): return b'\x80' + b'\x00'*((55-len(message))%64) + struct.pack('>Q',len(message)*8)

def length_extension_attack(message,mac,text_to_add):
    print('[=] performing length extension attack')
    registers = struct.unpack('>5I',unhexlify(mac))
    for key_length in range(50):
        glue_padding = get_mac_padding(b'A'*key_length+message)
        extended_message = message + glue_padding + text_to_add
        new_hash = my_sha1(text_to_add,(key_length+len(extended_message))*8,registers[0],registers[1],registers[2],registers[3],registers[4])
        if new_hash == sha1_mac(extended_message,key): return extended_message,new_hash

key = key_gen(random.randint(1,50))
message = b"comment1=cooking%20MCs;userdata=foo;comment2=%20like%20a%20pound%20of%20bacon"
mac = sha1_mac(message,key)
extended_message,new_hash = length_extension_attack(message,mac,b';admin=true')
print("[+] done! new message:",extended_message)
print("[+] forged hash:",new_hash)
print("[+] checking signature:",sha1_mac(extended_message,key))

[=] performing length extension attack
[+] done! new message: b'comment1=cooking%20MCs;userdata=foo;comment2=%20like%20a%20pound%20of%20bacon\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03(;admin=true'
[+] forged hash: 4c7d63e8d7afcee09d838266f2fcec8f8ce1d4e3
[+] checking signature: 4c7d63e8d7afcee09d838266f2fcec8f8ce1d4e3


### 30. Break an MD4 keyed MAC using length extension

In [33]:
from Crypto.Hash import MD4

def my_md4(message,ml=None,h=[0x67452301,0xEFCDAB89,0x98BADCFE,0x10325476]):
    if not ml: ml = len(message) * 8
    message += b"\x80"
    message += b"\x00" * (-(len(message)+8)%64)
    message += struct.pack("<Q", ml)
    chunks = [message[i:i+64] for i in range(0,len(message),64)]

    for chunk in chunks:
        X, temp_h = list(struct.unpack("<16I", chunk)), h.copy()

        Xi = [3, 7, 11, 19]
        for n in range(16):
            i, j, k, l = map(lambda x: x % 4, range(-n, -n + 4))
            K, S = n, Xi[n % 4]
            hn = temp_h[i] + ((temp_h[j] & temp_h[k]) | (~temp_h[j] & temp_h[l])) + X[K]
            temp_h[i] = leftrotate(hn&0xFFFFFFFF,S,32)

        Xi = [3, 5, 9, 13]
        for n in range(16):
            i, j, k, l = map(lambda x: x % 4, range(-n, -n + 4))
            K, S = n % 4 * 4 + n // 4, Xi[n % 4]
            hn = temp_h[i] + ((temp_h[j] & temp_h[k]) | (temp_h[j] & temp_h[l]) | (temp_h[k] & temp_h[l])) + X[K] + 0x5A827999
            temp_h[i] = leftrotate(hn&0xFFFFFFFF,S,32)

        Xi = [3, 9, 11, 15]
        Ki = [0, 8, 4, 12, 2, 10, 6, 14, 1, 9, 5, 13, 3, 11, 7, 15]
        for n in range(16):
            i, j, k, l = map(lambda x: x % 4, range(-n, -n + 4))
            K, S = Ki[n], Xi[n % 4]
            hn = temp_h[i] + (temp_h[j] ^ temp_h[k] ^ temp_h[l]) + X[K] + 0x6ED9EBA1
            temp_h[i] = leftrotate(hn&0xFFFFFFFF,S,32)
            
        h = [((v + n) & 0xFFFFFFFF) for v, n in zip(h, temp_h)]
        
    return "".join(f"{value:02x}" for value in struct.pack("<4L", *h))

def md4_mac(message,key): return my_md4(key+message)
def get_md4_padding(message): return b'\x80' + b'\x00'*((55-len(message))%64) + struct.pack('<Q',len(message)*8)

def md4_length_extension_attack(message,mac,text_to_add):
    print('[=] performing length extension attack')
    registers = struct.unpack('<4L',unhexlify(mac))
    for key_length in range(50):
        glue_padding = get_md4_padding(b'A'*key_length+message)
        extended_message = message + glue_padding + text_to_add
        new_hash = my_md4(text_to_add,(key_length+len(extended_message))*8,[registers[0],registers[1],registers[2],registers[3]])
        if new_hash == md4_mac(extended_message,key): return extended_message,new_hash

print("MD4 IMPLEMENTATION")
print("==================")
key = key_gen(random.randint(1,50))
test_message = b'message to test'
print('message:',test_message)
print('md4 hash:', my_md4(test_message))
h = MD4.new()
h.update(test_message)
print('check md4 implementation:', my_md4(test_message) == h.hexdigest())

mac = md4_mac(test_message,key)
print("\nMAC:",mac)
print("authenticate right message: ", mac == md4_mac(test_message,key))
print("authenticate wrong message: ", mac == md4_mac(key_gen(len(test_message)),key))

print("\n\nLENGTH EXTENSION ATTACK")
print("==================")
message = b"comment1=cooking%20MCs;userdata=foo;comment2=%20like%20a%20pound%20of%20bacon"
mac = md4_mac(message,key)
extended_message,new_hash = md4_length_extension_attack(message,mac,b';admin=true')
print("[+] done! new message:",extended_message)
print("[+] constructed hash:",new_hash)
print("[+] check signature:",md4_mac(extended_message,key))


MD4 IMPLEMENTATION
message: b'message to test'
md4 hash: ec497d2526590be57f70fb4cf25dd879
check md4 implementation: True

MAC: 120b2dde6191de6ad7e43be0be1ba340
authenticate right message:  True
authenticate wrong message:  False


LENGTH EXTENSION ATTACK
[=] performing length extension attack
[+] done! new message: b'comment1=cooking%20MCs;userdata=foo;comment2=%20like%20a%20pound%20of%20bacon\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc0\x03\x00\x00\x00\x00\x00\x00;admin=true'
[+] constructed hash: 6e7842d128924bbaf8b2dbae270f5aeb
[+] check signature: 6e7842d128924bbaf8b2dbae270f5aeb


### 31. Implement and break HMAC-SHA1 with an artificial timing leak

In [36]:
import hmac
import time

def my_hmac(message,key,hash,block_size):
    if len(key) > block_size: key = hash(key)
    if len(key) < block_size: key += b"\x00"*(block_size-len(key))
    o_key_pad = keyed_xor(b'\x5c'*block_size,key)
    i_key_pad = keyed_xor(b'\x36'*block_size,key)
    return hash(o_key_pad + unhexlify(hash(i_key_pad+message)))

print("HMAC IMPLEMENTATION")
print("==================")
key = key_gen(random.randint(1,50))
test_message = b'message to test'
print('message:',test_message)
print('hmac hash:', my_hmac(test_message,key,my_sha1,64))
h = hmac.new(key,digestmod=sha1)
h.update(test_message)
print('check hmac implementation:', my_hmac(test_message,key,my_sha1,64) == hexlify(h.digest()).decode())

def server(url,verbose=0,fast=1):
    parsed = {pair.split("=")[0]:pair.split("=")[1] for pair in url.split("?")[1].split("&")}
    if verbose: 
        print("file:",parsed['file'])
        print("signature:",parsed['signature'])
    return 200 if insecure_compare(my_hmac(parsed['file'].encode(),key,my_sha1,64),parsed['signature'],fast) else 500

def insecure_compare(s1,s2,fast=0): 
    if fast: return s1==s2
    for x,y in zip(s1,s2):
        if x!=y: return False
        time.sleep(sleeptime)
    return True

print("\n\nSIMULATED SERVER")
print("==================")
print('testing with right hmac')
filename = "testfile"
response = server('http://localhost:9000/test?file={}&signature='.format(filename)+my_hmac(filename.encode(),key,my_sha1,64),1)
print("server response:", response)
print('\ntesting with wrong hmac')
response = server('http://localhost:9000/test?file={}&signature='.format(filename+'a')+my_hmac(filename.encode(),key,my_sha1,64),1)
print("server response:", response)

def get_next_char(known_chars,filename):
    chars = 'abcdefghijklmnopqrstuvwxyz0123456789'
    times = []
    for char in chars:
        start = time.time()
        for _ in range(num_rounds): response = server('http://localhost:9000/test?file={}&signature={}'.format(filename,known_chars+char+'00000'),fast=0)
        end = time.time()
        times.append(end - start)
    print(chars[times.index(max(times))],end="")
    return chars[times.index(max(times))]    
        
def attack_server(filename,max_length):
    print("[+] attacking server for filename \'{}\'".format(filename))
    print("[=] true hmac: ",my_hmac(filename.encode(),key,my_sha1,64))
    print("[=] hmac found: ",end="")
    answer = ''
    chars = 'abcdefghijklmnopqrstuvwxyz0123456789'
    timeout = 0
    while True:
        answer += get_next_char(answer,filename)       
        response = server('http://localhost:9000/test?file={}&signature={}'.format(filename,answer))
        if response == 200: print("\n[+] done! got succesful response"); break
        timeout += 1
        if timeout >= max_length: print("\n[=] done! max length reached"); break

print("\n\nTIME-BASED ATTACK ON HMAC SHA1")
print("==================")
filename = "longdelayattack" 
sleeptime = .5
num_rounds = 1
attack_server(filename,50)

HMAC IMPLEMENTATION
message: b'message to test'
hmac hash: 598ac5505932fd71e200eab00f9cd61633e05f32
check hmac implementation: True


SIMULATED SERVER
testing with right hmac
file: testfile
signature: 7a45a0fc6e7c36d43e16d33e22052f2f793cab1a
server response: 200

testing with wrong hmac
file: testfilea
signature: 7a45a0fc6e7c36d43e16d33e22052f2f793cab1a
server response: 500


TIME-BASED ATTACK ON HMAC SHA1
[+] attacking server for filename 'longdelayattack'
[=] true hmac:  60cfcf8e6bd29cd640ad51bb7ed4518144851e67
[=] hmac found: 60cfcf8e6bd29cd640ad51bb7ed4518144851e67

[+] done! got succesful response


### 32. Break HMAC-SHA1 with a slightly less artificial timing leak

In [37]:
filename = "shortdelayattack" 
sleeptime = .01
num_rounds = 100
attack_server(filename,50)

[+] attacking server for filename 'shortdelayattack'
[=] true hmac:  294d1eca54134720265486cc5f1293f7a8036e02
[=] hmac found: 294d1eca54134720265486cc5f1293f7a8036e02

[+] done! got succesful response


### 33. Implement Diffie-Hellman

In [38]:
def modexp(b,e,m):
    x = 1
    while e>0: (b,e,x) = (b*b%m,e//2,b*x%m if e%2 else x)
    return x

def DH(p,g):
    a,b = random.randint(1,p),random.randint(1,p)
    A,B = modexp(g,a,p),modexp(g,b,p)
    return modexp(A,b,p),modexp(B,a,p)

print("SIMPLE DH")
print("==================")
s1,s2 = DH(37,5)
print("session key:",s1)
print("check session key:",s1==s2)

print("\n\nMORE REALISTIC DH")
print("==================")
p = 0xffffffffffffffffc90fdaa22168c234c4c6628b80dc1cd129024e088a67cc74020bbea63b139b22514a08798e3404ddef9519b3cd3a431b302b0a6df25f14374fe1356d6d51c245e485b576625e7ec6f44c42e9a637ed6b0bff5cb6f406b7edee386bfb5a899fa5ae9f24117c4b1fe649286651ece45b3dc2007cb8a163bf0598da48361c55d39a69163fa8fd24cf5f83655d23dca3ad961c62f356208552bb9ed529077096966d670c354e4abc9804f1746c08ca237327ffffffffffffffff
s1,s2 = DH(p,2)
print("session key:",s1)
print("check session key:",s1==s2)

SIMPLE DH
session key: 9
check session key: True


MORE REALISTIC DH
session key: 2211659596161325055576465739799480237584334291397148472199379047272329862141738750932309593049713936750463414048517966138118360774535117652546349853420296759527130691705685299479287643360500260916698755481295703470862947293604636877716317276784167255630279110768578470007456953336294697798624827660643326041159120405772041501902509868598709290107498761885442538629092736737091573653903741111209569875984149887545943470090502039922584192326504621251831056979180066
check session key: True


### 34. Implement a MITM key-fixing attack on Diffie-Hellman with parameter injection

In [39]:
p = 0xffffffffffffffffc90fdaa22168c234c4c6628b80dc1cd129024e088a67cc74020bbea63b139b22514a08798e3404ddef9519b3cd3a431b302b0a6df25f14374fe1356d6d51c245e485b576625e7ec6f44c42e9a637ed6b0bff5cb6f406b7edee386bfb5a899fa5ae9f24117c4b1fe649286651ece45b3dc2007cb8a163bf0598da48361c55d39a69163fa8fd24cf5f83655d23dca3ad961c62f356208552bb9ed529077096966d670c354e4abc9804f1746c08ca237327ffffffffffffffff
g = 2
message = 'Hello Bob'

class Alice:
    
    s = None
    a = None
    
    def __init__(self,p,g,msg):
        self.p = p
        self.g = g
        self.msg = msg
    
    def negotiate(self): 
        self.a = random.randint(1,self.p)
        A = modexp(self.g,self.a,self.p)
        return self.p,self.g,A
    
    def send(self,B):
        self.s = my_sha1(hex(modexp(B,self.a,self.p)))[:16].encode()
        iv = key_gen(16)
        ciphertext = CBC_encrypt(self.msg,iv,self.s)
        return ciphertext,iv
    
    def verify(self,ciphertext,iv): return remove_padding(CBC_decrypt(ciphertext,iv,self.s))[1].decode()
    
    
class Bob:
    
    s = None
    
    def __init__(self): pass
    
    def negotiate(self,p,g,A): 
        self.p = p
        self.g = g
        self.b = random.randint(1,self.p)
        self.B = modexp(self.g,self.b,self.p)
        self.s = my_sha1(hex(modexp(A,self.b,self.p)))[:16].encode()
        return self.B
    
    def receive(self,ciphertext,iv):
        plaintext = CBC_decrypt(ciphertext,iv,self.s)
        iv = key_gen(16)
        ciphertext = CBC_encrypt(plaintext,iv,self.s)
        return ciphertext,iv,remove_padding(plaintext)[1].decode()
    
    
print("NORMAL DHKE")
print("==================")
alice = Alice(p,g,message)
bob = Bob()
p,g,A = alice.negotiate()
print("ALICE NEGOTIATE:\np = {}\ng = {}\nA = {}".format(p,g,A))
B = bob.negotiate(p,g,A)
print("\nBOB NEGOTIATE:\nB = {}".format(B))
ciphertext,iv = alice.send(B)
print("\nALICE SEND:\nciphertext = {}\niv = {}".format(ciphertext,iv))
ciphertext,iv,plaintext = bob.receive(ciphertext,iv)
print("\nBOB VERIFY: plaintext = {}".format(plaintext))
print("\nBOB SEND:\nciphertext = {}\niv = {}".format(ciphertext,iv))
plaintext = alice.verify(ciphertext,iv)
print("\nALICE VERIFY: plaintext = {}".format(plaintext))

NORMAL DHKE
ALICE NEGOTIATE:
p = 2410312426921032588552076022197566074856950548502459942654116941958108831682612228890093858261341614673227141477904012196503648957050582631942730706805009223062734745341073406696246014589361659774041027169249453200378729434170325843778659198143763193776859869524088940195577346119843545301547043747207749969763750084308926339295559968882457872412993810129130294592999947926365264059284647209730384947211681434464714438488520940127459844288859336526896320919633919
g = 2
A = 1728423222770010344158587367328716559907278150162176332826949568988030706563081010479911735244937170098599727877687494465690925811146009538998370668966144891661354581739010922523292769741456668758338657266037118059928852279436398578192648199412812486306882925764682137969028207464436013317215186929220326885490208434290535683245409759668589976534485591740548733601991449638870692596025015195135152660936771071837916368623380929768604610388059736171839472167971353

BOB NEGOTIATE:
B = 144483770

In [40]:
p = 0xffffffffffffffffc90fdaa22168c234c4c6628b80dc1cd129024e088a67cc74020bbea63b139b22514a08798e3404ddef9519b3cd3a431b302b0a6df25f14374fe1356d6d51c245e485b576625e7ec6f44c42e9a637ed6b0bff5cb6f406b7edee386bfb5a899fa5ae9f24117c4b1fe649286651ece45b3dc2007cb8a163bf0598da48361c55d39a69163fa8fd24cf5f83655d23dca3ad961c62f356208552bb9ed529077096966d670c354e4abc9804f1746c08ca237327ffffffffffffffff
g = 2
message = 'Hello Bob'

class MITM:
   
    def __init__(self): pass
    
    def get_params(self,p=None,g=None,A=None,B=None):
        if p: self.p = p
        return self.p
    
    def make_key(self): self.key = my_sha1(hex(0))[:16].encode()
        
    def relay_and_decrypt(self,ciphertext,iv):
        plaintext = CBC_decrypt(ciphertext,iv,self.key)
        return remove_padding(plaintext)[1].decode(),ciphertext,iv


print("MITM ATTACK")
print("==================")
alice = Alice(p,g,message)
bob = Bob()
mitm = MITM()
p,g,A = alice.negotiate()
print("ALICE NEGOTIATE:\np = {}\ng = {}\nA = {}".format(p,g,A))
p = mitm.get_params(p=p,g=g,A=A)
print("\nMITM FORGE:\np = {}\ng = {}\nA = {}".format(p,g,p))
B = bob.negotiate(p,g,A)
print("\nBOB NEGOTIATE:\nB = {}".format(B))
p = mitm.get_params(B=B)
print("\nMITM FORGE:\nB = {}".format(p))
mitm.make_key()
ciphertext,iv = alice.send(p)
print("\nALICE SEND:\nciphertext = {}\niv = {}".format(ciphertext,iv))
plaintext,ciphertext,iv = mitm.relay_and_decrypt(ciphertext,iv)
print("\nMITM DECRYPT: plaintext = {}".format(plaintext))

MITM ATTACK
ALICE NEGOTIATE:
p = 2410312426921032588552076022197566074856950548502459942654116941958108831682612228890093858261341614673227141477904012196503648957050582631942730706805009223062734745341073406696246014589361659774041027169249453200378729434170325843778659198143763193776859869524088940195577346119843545301547043747207749969763750084308926339295559968882457872412993810129130294592999947926365264059284647209730384947211681434464714438488520940127459844288859336526896320919633919
g = 2
A = 1420863370350306958616110191053557490188398715943940644503685702469254257646942845268874085276323433217776658117042682641230668045247527675646785058619333496877800790934433530436813439884297502984713795850809918465381744683822247858277299951300084160722925610928316088545038800866334360391329970831921726821101341580545025668696588946496378056398793066945679574000884760864888039847078695095416193840659985575288752662208211059253368607413994054315542844751620613

MITM FORGE:
p = 241031242692

### 35. Implement DH with negotiated groups, and break with malicious "g" parameters

In [41]:
p = 0xffffffffffffffffc90fdaa22168c234c4c6628b80dc1cd129024e088a67cc74020bbea63b139b22514a08798e3404ddef9519b3cd3a431b302b0a6df25f14374fe1356d6d51c245e485b576625e7ec6f44c42e9a637ed6b0bff5cb6f406b7edee386bfb5a899fa5ae9f24117c4b1fe649286651ece45b3dc2007cb8a163bf0598da48361c55d39a69163fa8fd24cf5f83655d23dca3ad961c62f356208552bb9ed529077096966d670c354e4abc9804f1746c08ca237327ffffffffffffffff
g = 2
message = 'Hello Bob'

class Alice:

    def __init__(self,p,g,msg):
        self.p = p
        self.g = g
        self.msg = msg
    
    def negotiate_group(self): return self.p,self.g
    
    def negotiate_key(self): 
        self.a = random.randint(1,self.p)
        A = modexp(self.g,self.a,self.p)
        return A
    
    def send(self,B):
        self.s = my_sha1(hex(modexp(B,self.a,self.p)))[:16].encode()
        iv = key_gen(16)
        ciphertext = CBC_encrypt(self.msg,iv,self.s)
        return ciphertext,iv
    
    def verify(self,ciphertext,iv): return remove_padding(CBC_decrypt(ciphertext,iv,self.s))[1].decode()
    
    
class Bob:
  
    def __init__(self): pass
    
    def negotiate_group(self,p,g):
        self.p = p
        self.g = g
        self.b = random.randint(1,self.p)
        self.B = modexp(self.g,self.b,self.p)
        return 'OK'
        
    def negotiate_key(self,A): 
        self.s = my_sha1(hex(modexp(A,self.b,self.p)))[:16].encode()
        return self.B
    
    def receive(self,ciphertext,iv):
        plaintext = CBC_decrypt(ciphertext,iv,self.s)
        iv = key_gen(16)
        ciphertext = CBC_encrypt(plaintext,iv,self.s)
        return ciphertext,iv,remove_padding(plaintext)[1].decode()
    
    
print("NORMAL DHKE")
print("==================")
alice = Alice(p,g,message)
bob = Bob()
p,g = alice.negotiate_group()
print("ALICE NEGOTIATE GROUP:\np = {}\ng = {}".format(p,g,A))
reply = bob.negotiate_group(p,g)
print("\nBOB REPLY: {}".format(reply))
A = alice.negotiate_key()
print("\nALICE NEGOTIATE KEY:\nA = {}".format(A))
B = bob.negotiate_key(A)
print("\nBOB NEGOTIATE KEY:\nB = {}".format(B))
ciphertext,iv = alice.send(B)
print("\nALICE SEND:\nciphertext = {}\niv = {}".format(ciphertext,iv))
ciphertext,iv,plaintext = bob.receive(ciphertext,iv)
print("\nBOB VERIFY: plaintext = {}".format(plaintext))
print("\nBOB SEND:\nciphertext = {}\niv = {}".format(ciphertext,iv))
plaintext = alice.verify(ciphertext,iv)
print("\nALICE VERIFY: plaintext = {}".format(plaintext))

NORMAL DHKE
ALICE NEGOTIATE GROUP:
p = 2410312426921032588552076022197566074856950548502459942654116941958108831682612228890093858261341614673227141477904012196503648957050582631942730706805009223062734745341073406696246014589361659774041027169249453200378729434170325843778659198143763193776859869524088940195577346119843545301547043747207749969763750084308926339295559968882457872412993810129130294592999947926365264059284647209730384947211681434464714438488520940127459844288859336526896320919633919
g = 2

BOB REPLY: OK

ALICE NEGOTIATE KEY:
A = 138023444396878905143108501752220841457854728286663866597066467012182732116033499024018654610013334825573800527761790253939025098760426079422616162277424653296699347828013445396774246092166553136041928199234478111548814889554923588720495095237610667129853683787191594289030399891600933154875423088397170503854703530226923648675522169837676585894917686540853037894914408933492220825173874497466753164170528955567270223922061588341175957053286261938809

In [42]:
p = 0xffffffffffffffffc90fdaa22168c234c4c6628b80dc1cd129024e088a67cc74020bbea63b139b22514a08798e3404ddef9519b3cd3a431b302b0a6df25f14374fe1356d6d51c245e485b576625e7ec6f44c42e9a637ed6b0bff5cb6f406b7edee386bfb5a899fa5ae9f24117c4b1fe649286651ece45b3dc2007cb8a163bf0598da48361c55d39a69163fa8fd24cf5f83655d23dca3ad961c62f356208552bb9ed529077096966d670c354e4abc9804f1746c08ca237327ffffffffffffffff
g = 2
message = 'Hello Bob'

class MITM:
    
    key = None
    
    def __init__(self): pass
    
    def relay_group(self,p,g): return p,1
    def make_key(self): self.key = my_sha1(hex(1))[:16].encode()
        
    def relay_and_decrypt(self,ciphertext,iv):
        plaintext = CBC_decrypt(ciphertext,iv,self.key)
        return remove_padding(plaintext)[1].decode(),ciphertext,iv

print("MITM g=1 ATTACK")
print("==================")
alice = Alice(p,g,message)
bob = Bob()
mitm = MITM()
p,g = alice.negotiate_group()
print("ALICE NEGOTIATE GROUP:\np = {}\ng = {}".format(p,g,A))
p,g = mitm.relay_group(p,g)
print("\nMITM FORGE:\np = {}\ng = {}".format(p,g,))
reply = bob.negotiate_group(p,g)
print("\nBOB REPLY: {}".format(reply))
A = alice.negotiate_key()
print("\nALICE NEGOTIATE KEY:\nA = {}".format(A))
B = bob.negotiate_key(A)
print("\nBOB NEGOTIATE KEY:\nB = {}".format(B))
mitm.make_key()
ciphertext,iv = alice.send(B)
print("\nALICE SEND:\nciphertext = {}\niv = {}".format(ciphertext,iv))
plaintext,ciphertext,iv = mitm.relay_and_decrypt(ciphertext,iv)
print("\nMITM DECRYPT: plaintext = {}".format(plaintext))

MITM g=1 ATTACK
ALICE NEGOTIATE GROUP:
p = 2410312426921032588552076022197566074856950548502459942654116941958108831682612228890093858261341614673227141477904012196503648957050582631942730706805009223062734745341073406696246014589361659774041027169249453200378729434170325843778659198143763193776859869524088940195577346119843545301547043747207749969763750084308926339295559968882457872412993810129130294592999947926365264059284647209730384947211681434464714438488520940127459844288859336526896320919633919
g = 2

MITM FORGE:
p = 2410312426921032588552076022197566074856950548502459942654116941958108831682612228890093858261341614673227141477904012196503648957050582631942730706805009223062734745341073406696246014589361659774041027169249453200378729434170325843778659198143763193776859869524088940195577346119843545301547043747207749969763750084308926339295559968882457872412993810129130294592999947926365264059284647209730384947211681434464714438488520940127459844288859336526896320919633919
g = 1


In [43]:
p = 0xffffffffffffffffc90fdaa22168c234c4c6628b80dc1cd129024e088a67cc74020bbea63b139b22514a08798e3404ddef9519b3cd3a431b302b0a6df25f14374fe1356d6d51c245e485b576625e7ec6f44c42e9a637ed6b0bff5cb6f406b7edee386bfb5a899fa5ae9f24117c4b1fe649286651ece45b3dc2007cb8a163bf0598da48361c55d39a69163fa8fd24cf5f83655d23dca3ad961c62f356208552bb9ed529077096966d670c354e4abc9804f1746c08ca237327ffffffffffffffff
g = 2
message = 'Hello Bob'

class MITM:
    
    key = None
    
    def __init__(self): pass
    
    def relay_group(self,p,g): return p,p
    def make_key(self): self.key = my_sha1(hex(0))[:16].encode()
        
    def relay_and_decrypt(self,ciphertext,iv):
        plaintext = CBC_decrypt(ciphertext,iv,self.key)
        return remove_padding(plaintext)[1].decode(),ciphertext,iv

print("MITM g=p ATTACK")
print("==================")
alice = Alice(p,g,message)
bob = Bob()
mitm = MITM()
p,g = alice.negotiate_group()
print("ALICE NEGOTIATE GROUP:\np = {}\ng = {}".format(p,g,A))
p,g = mitm.relay_group(p,g)
print("\nMITM FORGE:\np = {}\ng = {}".format(p,g,))
reply = bob.negotiate_group(p,g)
print("\nBOB REPLY: {}".format(reply))
A = alice.negotiate_key()
print("\nALICE NEGOTIATE KEY:\nA = {}".format(A))
B = bob.negotiate_key(A)
print("\nBOB NEGOTIATE KEY:\nB = {}".format(B))
mitm.make_key()
ciphertext,iv = alice.send(B)
print("\nALICE SEND:\nciphertext = {}\niv = {}".format(ciphertext,iv))
plaintext,ciphertext,iv = mitm.relay_and_decrypt(ciphertext,iv)
print("\nMITM DECRYPT: plaintext = {}".format(plaintext))

MITM g=p ATTACK
ALICE NEGOTIATE GROUP:
p = 2410312426921032588552076022197566074856950548502459942654116941958108831682612228890093858261341614673227141477904012196503648957050582631942730706805009223062734745341073406696246014589361659774041027169249453200378729434170325843778659198143763193776859869524088940195577346119843545301547043747207749969763750084308926339295559968882457872412993810129130294592999947926365264059284647209730384947211681434464714438488520940127459844288859336526896320919633919
g = 2

MITM FORGE:
p = 2410312426921032588552076022197566074856950548502459942654116941958108831682612228890093858261341614673227141477904012196503648957050582631942730706805009223062734745341073406696246014589361659774041027169249453200378729434170325843778659198143763193776859869524088940195577346119843545301547043747207749969763750084308926339295559968882457872412993810129130294592999947926365264059284647209730384947211681434464714438488520940127459844288859336526896320919633919
g = 24

In [50]:
p = 0xffffffffffffffffc90fdaa22168c234c4c6628b80dc1cd129024e088a67cc74020bbea63b139b22514a08798e3404ddef9519b3cd3a431b302b0a6df25f14374fe1356d6d51c245e485b576625e7ec6f44c42e9a637ed6b0bff5cb6f406b7edee386bfb5a899fa5ae9f24117c4b1fe649286651ece45b3dc2007cb8a163bf0598da48361c55d39a69163fa8fd24cf5f83655d23dca3ad961c62f356208552bb9ed529077096966d670c354e4abc9804f1746c08ca237327ffffffffffffffff
g = 2
message = 'Hello Bob'

class MITM:
    
    key1 = None
    key2 = None
    p = None
    
    def __init__(self): pass
    
    def relay_group(self,p,g): 
        self.p = p
        return p,-1
    
    def make_key(self): 
        self.key1 = my_sha1(hex(1))[:16].encode()
        self.key2 = my_sha1(hex(p-1))[:16].encode()
        
    def relay_and_decrypt(self,ciphertext,iv):
        plaintext1 = CBC_decrypt(ciphertext,iv,self.key1)
        plaintext2 = CBC_decrypt(ciphertext,iv,self.key2)
        return plaintext1,plaintext2,ciphertext,iv

print("MITM g=p-1 ATTACK")
print("==================")
alice = Alice(p,g,message)
bob = Bob()
mitm = MITM()
p,g = alice.negotiate_group()
print("ALICE NEGOTIATE GROUP:\np = {}\ng = {}".format(p,g,A))
p,g = mitm.relay_group(p,g)
print("\nMITM FORGE:\np = {}\ng = {}".format(p,g,))
reply = bob.negotiate_group(p,g)
print("\nBOB REPLY: {}".format(reply))
A = alice.negotiate_key()
print("\nALICE NEGOTIATE KEY:\nA = {}".format(A))
B = bob.negotiate_key(A)
print("\nBOB NEGOTIATE KEY:\nB = {}".format(B))
mitm.make_key()
ciphertext,iv = alice.send(B)
print("\nALICE SEND:\nciphertext = {}\niv = {}".format(ciphertext,iv))
plaintext1,plaintext2,ciphertext,iv = mitm.relay_and_decrypt(ciphertext,iv)
if remove_padding(plaintext1)[0] and remove_padding(plaintext2)[0]: print("\nMITM DECRYPT:\npossible plaintext 1 = {}\npossible plaintext 2 = {}".format(remove_padding(plaintext1)[1].decode(),remove_padding(plaintext2)[1].decode()))
elif remove_padding(plaintext1)[0]: print("\nMITM DECRYPT:\nplaintext = {}".format(remove_padding(plaintext1)[1].decode()))
else: print("\nMITM DECRYPT:\nplaintext = {}".format(remove_padding(plaintext2)[1].decode()))

MITM g=p-1 ATTACK
ALICE NEGOTIATE GROUP:
p = 2410312426921032588552076022197566074856950548502459942654116941958108831682612228890093858261341614673227141477904012196503648957050582631942730706805009223062734745341073406696246014589361659774041027169249453200378729434170325843778659198143763193776859869524088940195577346119843545301547043747207749969763750084308926339295559968882457872412993810129130294592999947926365264059284647209730384947211681434464714438488520940127459844288859336526896320919633919
g = 2

MITM FORGE:
p = 2410312426921032588552076022197566074856950548502459942654116941958108831682612228890093858261341614673227141477904012196503648957050582631942730706805009223062734745341073406696246014589361659774041027169249453200378729434170325843778659198143763193776859869524088940195577346119843545301547043747207749969763750084308926339295559968882457872412993810129130294592999947926365264059284647209730384947211681434464714438488520940127459844288859336526896320919633919
g = 

### 36. Implement Secure Remote Password (SRP)

In [51]:
from hashlib import sha256,pbkdf2_hmac

N = 0xffffffffffffffffc90fdaa22168c234c4c6628b80dc1cd129024e088a67cc74020bbea63b139b22514a08798e3404ddef9519b3cd3a431b302b0a6df25f14374fe1356d6d51c245e485b576625e7ec6f44c42e9a637ed6b0bff5cb6f406b7edee386bfb5a899fa5ae9f24117c4b1fe649286651ece45b3dc2007cb8a163bf0598da48361c55d39a69163fa8fd24cf5f83655d23dca3ad961c62f356208552bb9ed529077096966d670c354e4abc9804f1746c08ca237327ffffffffffffffff
g = 2
k = 3
I = "admin@mail.com"
P = "P@ssw0rd!123"

class Server:
    
    def __init__(self,N,g,k,I,P):
        self.N = N
        salt = str(random.randint(0,self.N))
        xH = sha256((salt+P).encode()).hexdigest()
        x = int(xH,16)
        self.salt = salt
        self.v = modexp(g,x,N)
        self.g = g
        self.k = k
        self.I = I
        self.P = P
    
    def negotiate_key(self):
        self.b = random.randint(0,self.N-1)
        self.B = (self.k*self.v + modexp(self.g,self.b,self.N)) % self.N
        return self.salt,self.B
    
    def validate_password(self,A,mac): 
        uH = sha256((format(A,'x')+format(self.B,'x')).encode()).hexdigest()
        u = int(uH,16)
        S = modexp(A*modexp(self.v,u,self.N),self.b,self.N)
        K = sha256(str(S).encode()).hexdigest()
        return pbkdf2_hmac('sha256',K.encode(),salt.encode(),1) == mac
    
        
class Client:
    
    def __init__(self,N,g,k,I,P):
        self.N = N
        self.g = g
        self.k = k
        self.I = I
        self.P = P
    
    def negotiate_key(self):
        self.a = random.randint(1,N)
        self.A = modexp(self.g,self.a,self.N)
        return self.I,self.A
    
    def send_password(self,B,salt): 
        uH = sha256((format(self.A,'x')+format(B,'x')).encode()).hexdigest()
        u = int(uH,16)
        xH = sha256((salt+self.P).encode()).hexdigest()
        x = int(xH,16)
        S = modexp(B - self.k * modexp(self.g,x,self.N),self.a + u * x,self.N)
        K = sha256(str(S).encode()).hexdigest()
        return pbkdf2_hmac('sha256',K.encode(),salt.encode(),1)

print("NORMAL SRP")
print("==================")
server = Server(N,g,k,I,P)
client = Client(N,g,k,I,P)
email,A = client.negotiate_key()
print("CLIENT NEGOTIATE KEY:\nemail = {}\nA = {}".format(email,A))
salt,B = server.negotiate_key()
print("\nSERVER NEGOTIATE KEY:\nsalt = {}\nB = {}".format(salt,B))
mac = client.send_password(B,salt)
print("\nCLIENT AUTHENTICATES:\ncleartext password = {}\nhashed password = {}".format(client.P,mac))
validation = server.validate_password(A,mac)
print("\nSERVER VALIDATION: {}".format(validation))
print("\n\nFAILED AUTHENTICATION")
print("==================")
server = Server(N,g,k,I,P)
client = Client(N,g,k,I,"different password")
email,A = client.negotiate_key()
print("CLIENT IDENTIFICATION:\nemail = {}".format(email))
salt,B = server.negotiate_key()
mac = client.send_password(B,salt)
print("\nCLIENT AUTHENTICATES:\ncleartext password = {}\nhashed password = {}".format(client.P,mac))
validation = server.validate_password(A,mac)
print("\nSERVER VALIDATION: {}".format(validation))

NORMAL SRP
CLIENT NEGOTIATE KEY:
email = admin@mail.com
A = 362549979850827220182253532110360904925742943515871355006611185695899300920830374894773625184326739678629889583609489039726147773195968204400163435741608074518526486998011314535882121658884914375190841061348477367187383350516111247077105503051137630274510739365607003899030609360675949422717576798413191173833674827843120581785067463110103690855624093960985229089366887580587142622821996832142642901230924466213191525302212383006138635915550100621342319822826199

SERVER NEGOTIATE KEY:
salt = 180098939320687925267023193520812096859567856622176116326177460656325338139440876467581389173279809438634361759340770624328847726029462513948815268879041168712129328213385089391122738014227904299741810679514028496160708359161394166570443951942670736264636996509843727825725367668065299160438590879430400889006850759629541650092181413188131933020394033447908187053728194151781335534732053466472707861352815741262596408503925983553623937735704559320

### 37. Break SRP with a zero key

In [52]:
N = 0xffffffffffffffffc90fdaa22168c234c4c6628b80dc1cd129024e088a67cc74020bbea63b139b22514a08798e3404ddef9519b3cd3a431b302b0a6df25f14374fe1356d6d51c245e485b576625e7ec6f44c42e9a637ed6b0bff5cb6f406b7edee386bfb5a899fa5ae9f24117c4b1fe649286651ece45b3dc2007cb8a163bf0598da48361c55d39a69163fa8fd24cf5f83655d23dca3ad961c62f356208552bb9ed529077096966d670c354e4abc9804f1746c08ca237327ffffffffffffffff
g = 2
k = 3
I = "admin@mail.com"
P = "P@ssw0rd!123"

class Server:
    
    def __init__(self,N,g,k,I,P):
        self.N = N
        salt = str(random.randint(0,self.N))
        xH = sha256((salt+P).encode()).hexdigest()
        x = int(xH,16)
        self.salt = salt
        self.v = modexp(g,x,N)
        self.g = g
        self.k = k
        self.I = I
        self.P = P
    
    def negotiate_key(self):
        self.b = random.randint(0,self.N-1)
        self.B = (self.k*self.v + modexp(self.g,self.b,self.N)) % self.N
        return self.salt,self.B
    
    def validate_password(self,A,mac): 
        uH = sha256((format(A,'x')+format(self.B,'x')).encode()).hexdigest()
        u = int(uH,16)
        S = modexp(A*modexp(self.v,u,self.N),self.b,self.N)
        K = sha256(str(S).encode()).hexdigest()
        return pbkdf2_hmac('sha256',K.encode(),salt.encode(),1) == mac
    
    
class Client:
    
    def __init__(self,N,g,k,I):
        self.N = N
        self.g = g
        self.k = k
        self.I = I
    
    def negotiate_key(self): return self.I,0
    
    def send_password(self,B,salt): 
        S = 0
        K = sha256(str(S).encode()).hexdigest()
        return pbkdf2_hmac('sha256',K.encode(),salt.encode(),1)

server = Server(N,g,k,I,P)
client = Client(N,g,k,I)
email,A = client.negotiate_key()
print("CLIENT NEGOTIATE KEY:\nemail = {}\nA = {}".format(email,A))
salt,B = server.negotiate_key()
print("\nSERVER NEGOTIATE KEY:\nsalt = {}\nB = {}".format(salt,B))
mac = client.send_password(B,salt)
print("\nCLIENT SENDS HASHED PASSWORD: K = {}".format(mac))
validation = server.validate_password(A,mac)
print("\nSERVER VALIDATION: {}".format(validation))

CLIENT NEGOTIATE KEY:
email = admin@mail.com
A = 0

SERVER NEGOTIATE KEY:
salt = 541498221262949561898859023116603078233107242649184361398432039454643441296078726829507003270618285811693415949295129293527437708819130584241664374085126140315057854501760080259727611493088064238047580898353774267098312068458287533679298769937583745273062389168832575377229719971037775238687025139929353824908220227321947260136822910339371974216522922664813219974345796255522961286556819348891977769741457488390979581546771776210175957533415390406194725892855526
B = 12493627715100991856774686542143107010241758440305999491495370439227865700695465500988713280613191288636623958110775737204177378359314438795389561593427576525636216628997808353595669381189817849784682878158088228784346867145595923334043005734545837273615479139632949019964794186118184050722395580903524251432923692709947789455902581121363093368431080290320717655027081165745190252152289473975703321606620910044506396743095887681921188692405751369489290

### 38. Offline dictionary attack on simplified SRP

In [53]:
password = "Password443"
wordlist = ["Password" + str(i) for i in range(1000)]
N = 0xffffffffffffffffc90fdaa22168c234c4c6628b80dc1cd129024e088a67cc74020bbea63b139b22514a08798e3404ddef9519b3cd3a431b302b0a6df25f14374fe1356d6d51c245e485b576625e7ec6f44c42e9a637ed6b0bff5cb6f406b7edee386bfb5a899fa5ae9f24117c4b1fe649286651ece45b3dc2007cb8a163bf0598da48361c55d39a69163fa8fd24cf5f83655d23dca3ad961c62f356208552bb9ed529077096966d670c354e4abc9804f1746c08ca237327ffffffffffffffff
g = 3
b = random.randint(1,1000)
B = modexp(g,b,N)
u = random.randint(1,1000)
salt = random.randint(1,N)

def hash_pass(password,N,B,u,salt): 
    a = random.randint(1,100)
    A = modexp(g,a,N)
    x = sha256((format(salt,'x') + password).encode()).hexdigest() 
    x = int(x,16)
    S = modexp(B,a + x*u,N)
    K = sha256(hex(S).encode()).hexdigest()
    return pbkdf2_hmac('sha256',K.encode(),(format(salt,'x') + password).encode(),1),A

def crack_mac(mac,wordlist,A,b,B,g):
    print('[=] dictionary attack on SRP')
    for test_pass in wordlist:
        x = sha256((format(salt,'x') + test_pass).encode()).hexdigest() 
        x = int(x,16)
        S = modexp(A*modexp(g,x*u,N),b,N)
        K = sha256(hex(S).encode()).hexdigest()
        test_mac = pbkdf2_hmac('sha256',K.encode(),(format(salt,'x') + password).encode(),1)
        if test_mac == mac: return test_pass
    return "not found"

mac,A = hash_pass(password,N,B,u,salt)
print("[=] password hash:",mac)
cracked = crack_mac(mac,wordlist,A,b,B,g)
print("[+] done! cracked password:",cracked)

[=] password hash: b'(&\x8c\x16\xe2\x8d\xeb\x1f\xbd\x04\x8f\xf6id\x99\xef\x0b\xec\xbf:\x91E\x08<\xbd\xdd\xb4\x17=\x04\x91\xdb'
[=] dictionary attack on SRP
[+] done! cracked password: Password443


### 39. Implement RSA

In [54]:
import math
import codecs
from Crypto.Util.number import getPrime

class RSA:    
    def __init__(self,e,key_length):
        self.e = e
        self.key_length = key_length
        et = 0
        while math.gcd(et,e) != 1:
            p = getPrime(key_length)
            q = getPrime(key_length)
            et = (p-1)*(q-1)//math.gcd(p-1,q-1)
        self.n = p * q
        self.d = pow(e,-1,et)
    
    def public_key(self): return [self.e,self.n]
    def encrypt(self,plaintext): 
        if isinstance(plaintext,str): plaintext = plaintext.encode()
        if isinstance(plaintext,bytes): plaintext = int.from_bytes(plaintext,'big')
        return modexp(plaintext,self.e,self.n)
    def decrypt(self,ciphertext): return modexp(ciphertext,self.d,self.n)

rsa = RSA(3,1024)
plaintext = "hello"
print("plaintext:",plaintext)
ciphertext = rsa.encrypt(plaintext)
print("ciphertext:",ciphertext)
print("decrypted:", codecs.decode(format(rsa.decrypt(ciphertext),'x').encode(),'hex').decode())

plaintext: hello
ciphertext: 90143305010218464651239068244550223
decrypted: hello


### 40. Implement an E=3 RSA Broadcast attack

In [55]:
def triple_encrypt(plaintext):
    pairs = []
    for _ in range(3):
        rsa = RSA(3,1024)
        pairs.append([rsa.public_key()[1],rsa.encrypt(plaintext)])
    return pairs

def CRT_solve(pairs): 
    print('[=] broadcast attack on e=3 RSA')
    prod = pairs[0][0]*pairs[1][0]*pairs[2][0]
    cube = (pow(pairs[1][0]*pairs[2][0],-1,pairs[0][0])*pairs[1][0]*pairs[2][0]*pairs[0][1])
    cube += (pow(pairs[0][0]*pairs[2][0],-1,pairs[1][0])*pairs[0][0]*pairs[2][0]*pairs[1][1])
    cube += (pow(pairs[1][0]*pairs[0][0],-1,pairs[2][0])*pairs[1][0]*pairs[0][0]*pairs[2][1])
    cube %= prod
    cube = int(pow(cube,1/3)) + 1
    return cube

plaintext = 'hello'
ciphertexts = triple_encrypt(plaintext)
print("[+] done! decrypted text:", codecs.decode(format(CRT_solve(ciphertexts),'x').encode(),'hex').decode())

[=] broadcast attack on e=3 RSA
[+] done! decrypted text: hello


### 41. Implement unpadded message recovery oracle

In [56]:
rsa = RSA(3,1024)
plaintext = 'hello'
ciphertext = rsa.encrypt(plaintext)
public_key = rsa.public_key()

def recovery_oracle_attack(ciphertext,public_key,oracle): 
    print('[=] performing recovery oracle attack')
    factor = modexp(2,public_key[0],public_key[1])
    modified_ciphertext = (factor*ciphertext) % public_key[1]
    modified_plaintext = rsa.decrypt(modified_ciphertext)
    plaintext = modified_plaintext*pow(2,-1,public_key[1]) %public_key[1]
    return codecs.decode(format(plaintext,'x').encode(),'hex').decode()

plaintext = recovery_oracle_attack(ciphertext,public_key,rsa)
print('[+] done! decrypted text: {}'.format(plaintext))

[=] performing recovery oracle attack
[+] done! decrypted text: hello



### 42. Bleichenbacher's e=3 RSA Attack

In [57]:
import re

ansi = '3021300906052b0e03021a05000414'

def nearest_cube_root(num):
    l,r = 0,num
    while l<r:
        x = ((l+r)//2)**3
        l,r = ((l+r)//2+1,r) if x < num else (l,(l+r)//2)
    return l-1

class RSA_DSA(RSA):
    def sign(self,message): return modexp(int(codecs.encode(target_string.encode(),'hex'),16),self.d,self.n)
    def verify(self,signature,message): 
        block_input = '000000' + format(modexp(signature,self.e,self.n),'x')
        m = re.compile('0001(ff)*00'+ansi+'(.{40})',re.DOTALL).search(block_input)
        if not m: return False
        return m[2] == my_sha1(message)
 
def signature_forger(dsa,target_string):
    print("[=] forging dsa signature for string:", target_string)
    hash = my_sha1(target_string)
    print("[=] target hash:",hash)
    N = rsa.public_key()[1]
    start_string = '0001'
    first_group = 1
    R = re.compile(start_string + '(ff)*00'+ansi+'(.{40})',re.DOTALL)
    while int(start_string,16) < N:
        start_string = '0001' + 'ff'*first_group + '00' + ansi + hash
        target_string = start_string
        while int(target_string,16) < N:
            root = nearest_cube_root(int(target_string,16))
            test_block = '000000' + format(modexp(root,3,N),'x')
            m = R.search(test_block)
            if m and m[2] == hash:
                print("[+] got a match!")
                return root
            target_string += 'f'
        first_group += 1
    print("[-] failed")
    print("[-]maybe try increasing n")
        
rsa = RSA_DSA(3,1024)
target_string = "hi mom"
signature = signature_forger(rsa,target_string)
print('[+] forged signature:',signature)
print("[+] forged block:",'000000' + format(modexp(signature,3,N),'x'))
print("[+] dsa verification:", rsa.verify(signature,target_string))

[=] forging dsa signature for string: hi mom
[=] target hash: 925a89b43f3caff507db0a86d20a2428007f10b6
[+] got a match!
[+] forged signature: 403936002991693898718319226857676803001521085440508927910622149420577689156740474161131190
[+] forged block: 0000001ff003021300906052b0e03021a05000414925a89b43f3caff507db0a86d20a2428007f10b6699e8640d7ec1736ad57272a5d309815b2169c2ce83c7d95127e61b1a3df76489159604c8e2e7ef5350c9d3bd0aaede4134af0caf46366974520693c215aefdb6e3a2291309d21304e518
[+] dsa verification: True


### 43. DSA key recovery from nonce

In [58]:
class DSA:
    
    def __init__(self,hashalg,p,q,g):
        self.hashalg = hashalg
        self.g = g
        self.p = p
        self.q = q
        self.x = random.randint(1,q-1)
        self.y = modexp(g,self.x,p)
        
    def get_public_key(self): return self.y
    def leak_subkey(self): return self.k,self.x
    
    def sign(self,message):
        r = s = 0
        while r*s == 0:
            k = random.randint(1,self.q-1)
            r = modexp(self.g,k,self.p) % self.q
            h = int(hexlify(self.hashalg(message).encode())) % self.q
            s = (h + self.x*r)*pow(k,-1,self.q) % self.q
            self.k = k
        return r,s
    
    def verify(self,r,s,message):
        if r >= self.q or s >= self.q: return False
        w = pow(s,-1,self.q)
        hash = int(hexlify(self.hashalg(message).encode()))%self.q
        u1 = (hash*w)%self.q
        u2 = (r*w)%self.q
        v = (modexp(self.g,u1,self.p)*modexp(self.y,u2,self.p))%self.p%self.q
        return v == r
 
def x_from_k(k,s,r,q,hash_in_dec): return (s*k-hash_in_dec)*pow(r,-1,q)%q
    
p = 0x800000000000000089e1855218a0e7dac38136ffafa72eda7859f2171e25e65eac698c1702578b07dc2a1076da241c76c62d374d8389ea5aeffd3226a0530cc565f3bf6b50929139ebeac04f48c3c84afb796d61e5a4f9a8fda812ab59494232c7d2b4deb50aa18ee9e132bfa85ac4374d7f9091abc3d015efc871a584471bb1
q = 0xf4f47f05794b256174bba6e9b396a7707e563c5b
g = 0x5958c9d3898b224b12672c0b98e06c60df923cb8bc999d119458fef538b8fa4046c8db53039db620c094c9fa077ef389b5322a559946a71903f990f1f7e0e025e2d7f7cf494aff1a0470f5b64c36b625a097f1651fe775323556fe00b3608c887892878480e99041be601a62166ca6894bdd41a7054ec89f756ba9fc95302291

print("DSA VERIFICATION")
print("==================")
dsa = DSA(my_sha1,p,q,g)
message = 'hello'
print("message:",message)
r,s = dsa.sign(message)
print("signature: r = {}; s = {}".format(r,s))
verification = dsa.verify(r,s,message)
print("verification:",verification)

print("\n\nKEY FROM NONCE VERIFICATION")
print("==================")
k,x = dsa.leak_subkey()
print("cracked key:",x_from_k(k,s,r,q,int(hexlify(my_sha1(message).encode()))))
print("true key:",x)

print("\n\nCRACK KEY FROM NONCE")
print("==================")

q = 0xf4f47f05794b256174bba6e9b396a7707e563c5b
hash = 0xd2d0714f014a9784047eaeccf956520045c45265
r = 548099063082341131477253921760299949438196259240
s = 857042759984254168557880549501802188789837994940
print('[=] brute force attack on private key')

for possible_k in range(0xFFFF):
    possible_x = x_from_k(possible_k,s,r,q,hash)
    if my_sha1(format(possible_x,'x')) == '0954edd5e0afe5542a4adf012611a91912a3ec16': 
        print("[+] private key found:",x)
        break

DSA VERIFICATION
message: hello
signature: r = 160949027415211423302633809544943524547567894368; s = 315982465048016018318741293839107862485390186478
verification: True


KEY FROM NONCE VERIFICATION
cracked key: 1249016559660585382409570458061896043190036481014
true key: 1249016559660585382409570458061896043190036481014


CRACK KEY FROM NONCE
[=] brute force attack on private key
[+] private key found: 1249016559660585382409570458061896043190036481014


### 44. DSA nonce recovery from repeated nonce

In [59]:
q = 0xf4f47f05794b256174bba6e9b396a7707e563c5b

def get_match(file):
    print('[=] looking for repeated nonce')
    data = [line.strip().split(': ')[1] for line in open(file,'r').readlines()]
    chunks = [data[n:n+4] for n in range(0,len(data),4)]
    rs = [chunk[2] for chunk in chunks]
    for n in range(len(rs)): 
        if rs.count(rs[n]) > 1: 
            print('[+] found a pair')
            return chunks[n], chunks[n+1:][rs[n+1:].index(rs[n])]
    return None
    
def k_from_match(msrm1,msrm2):
    m1,m2 = int(msrm1[3],16),int(msrm2[3],16)
    s1,s2 = int(msrm1[1]),int(msrm2[1])
    return ((m1-m2)*pow((s1-s2)%q,-1,q))%q
      
msrm1, msrm2 = get_match('files/44.txt')
k = k_from_match(msrm1,msrm2)
print("[+] cracked subkey:",k)
x = x_from_k(k,int(msrm1[1]),int(msrm1[2]),q,int(msrm1[3],16))
print("[+] recovered private key from subkey:",x)
print("[+] private key verification:", my_sha1(format(x,'x')) == 'ca8f6f7c66fa362d40760d135b763eb8527d3d52')

[=] looking for repeated nonce
[+] found a pair
[+] cracked subkey: 108994997653034620063305500641348549625
[+] recovered private key from subkey: 1379952329417023174824742221952501647027600451162
[+] private key verification: True



### 45. DSA parameter tampering

In [60]:
class weak_DSA:
    
    def __init__(self,hashalg,p,q,g):
        self.hashalg = hashalg
        self.g = g
        self.p = p
        self.q = q
        self.x = random.randint(1,q-1)
        self.y = modexp(g,self.x,p)
        
    def get_public_key(self): return self.y
    def leak_subkey(self): return self.k,self.x
    
    def sign(self,message):
        k = random.randint(1,self.q-1)
        r = modexp(self.g,k,self.p) % self.q
        h = int(hexlify(self.hashalg(message).encode())) % self.q
        s = (h + self.x*r)*pow(k,-1,self.q) % self.q
        self.k = k
        return r,s
    
    def verify(self,r,s,message):
        if r >= self.q or s >= self.q: return False
        w = pow(s,-1,self.q)
        hash = int(hexlify(self.hashalg(message).encode()))%self.q
        u1 = (hash*w)%self.q
        u2 = (r*w)%self.q
        v = (modexp(self.g,u1,self.p)*modexp(self.y,u2,self.p))%self.p%self.q
        return v == r
 

p = 0x800000000000000089e1855218a0e7dac38136ffafa72eda7859f2171e25e65eac698c1702578b07dc2a1076da241c76c62d374d8389ea5aeffd3226a0530cc565f3bf6b50929139ebeac04f48c3c84afb796d61e5a4f9a8fda812ab59494232c7d2b4deb50aa18ee9e132bfa85ac4374d7f9091abc3d015efc871a584471bb1
q = 0xf4f47f05794b256174bba6e9b396a7707e563c5b
message = "hello world"
wrong_message = "goodbye world"
print("DSA WITH g=0")
print("==================")
dsa = weak_DSA(my_sha1,p,q,0)
print("message:",message)
r,s = dsa.sign(message)
print("signature: r = {}; s = {}".format(r,s))
print("verification:",dsa.verify(r,s,message))
print("fake message:",wrong_message)
print("verification:",dsa.verify(r,s,wrong_message))

print("\n\nDSA WITH g=p+1")
print("==================")
dsa = weak_DSA(my_sha1,p,q,1)
z = random.randint(1,q-1)
y = dsa.y
r = modexp(y,z,p)%q
s = r*pow(z,-1,q)%q
print("magic signature: r = {}; s = {}".format(r,s))
print("message 1:",message)
print("verification:",dsa.verify(r,s,message))
print("message 2:",wrong_message)
print("verification:",dsa.verify(r,s,wrong_message))


DSA WITH g=0
message: hello world
signature: r = 0; s = 1390365007069888247122189939023373553274115933034
verification: True
fake message: goodbye world
verification: True


DSA WITH g=p+1
magic signature: r = 1; s = 831525787277319352857066059935890795773072379979
message 1: hello world
verification: True
message 2: goodbye world
verification: True


### 46. RSA parity oracle

In [61]:
from decimal import *

class oracle(RSA):
    def oracle(self,ciphertext):
        return self.decrypt(ciphertext)%2
    
def attack_rsa_oracle(ciphertext,oracle):
    print("[=] attacking rsa parity oracle")
    N = oracle.public_key()[1]
    factor = modexp(2,oracle.e,N)
    lower = Decimal(0)
    upper = Decimal(N)
    getcontext().prec = n_bits
    
    for i in range(n_bits):
        ciphertext = ciphertext*factor%N    
        if oracle.oracle(ciphertext): lower = (lower+upper)/2
        else: 
            upper = (lower+upper)/2
            try: print('[=] partially decrypted text: ', codecs.decode(format(int(upper),'x').encode(),'hex').decode())
            except: continue
    return int(upper)
 
n_bits = 1024
message = b64decode('VGhhdCdzIHdoeSBJIGZvdW5kIHlvdSBkb24ndCBwbGF5IGFyb3VuZCB3aXRoIHRoZSBGdW5reSBDb2xkIE1lZGluYQ==')
rsa = oracle(3,n_bits//2)
ciphertext = rsa.encrypt(message.decode())
plaintext = attack_rsa_oracle(ciphertext,rsa)
print("[+] done! recovered plaintext:", codecs.decode(format(plaintext,'x').encode(),'hex').decode())

[=] attacking rsa parity oracle
[=] partially decrypted text:  That's why I found you don't play around with the Funky Coldo|P*8
[=] partially decrypted text:  That's why I found you don't play around with the Funky Cold Mi6-
[=] partially decrypted text:  That's why I found you don't play around with the Funky Cold Medio
[=] partially decrypted text:  That's why I found you don't play around with the Funky Cold Medinl
[=] partially decrypted text:  That's why I found you don't play around with the Funky Cold Medinf
[=] partially decrypted text:  That's why I found you don't play around with the Funky Cold Medinc
[=] partially decrypted text:  That's why I found you don't play around with the Funky Cold Medina
[=] partially decrypted text:  That's why I found you don't play around with the Funky Cold Medina
[+] done! recovered plaintext: That's why I found you don't play around with the Funky Cold Medina


### 47. Bleichenbacher's PKCS 1.5 Padding Oracle (Simple Case)

In [62]:
class RSA_Padding_Oracle(RSA):
    def pad(self,data,total_length):
        pad_length = total_length-3-len(data)
        padded = b'\x00\x02'
        for _ in range(pad_length): padded += bytes([random.randint(1,255)])
        return padded + b'\x00' + data
    
    def pad_check(self,ciphertext_int):
        plaintext_int = oracle.decrypt(ciphertext_int)
        plaintext = plaintext_int.to_bytes((plaintext_int.bit_length()+7)//8+1, 'big')
        return plaintext[:2] == b'\x00\x02' and len(plaintext)==self.key_length//4
       
def union(M,low,high):
    to_remove = set()
    for (a,b) in M:
        if a <= high and b >= low: 
            low,high = min(low,a),max(high, b)
            to_remove.add((a,b))
    for pair in to_remove: M.remove(pair)
    M.append((low, high))
    
def attack_RSA_padding_oracle(oracle,ciphertext):

    print('[=] attacking padding oracle')
    B = 2**(oracle.key_length*2-16)
    e,n = oracle.public_key()
    M = [(2*B,3*B-1)]
    i = 1
    
    s = 1 
    while not oracle.pad_check((ciphertext*modexp(s,e,n))%n): s += 1
        
    while not (len(M) == 1 and M[0][0] == M[0][1]):
        
        if i == 1:
            s = n//(3*B)
            while not oracle.pad_check((ciphertext*modexp(s,e,n))%n): 
                s += 1
                if not (s-n//(3*B))%10000: print('[=] tried {} values for s'.format(s-n//(3*B)))
            print('[+] initialisation for s done')

        elif len(M) > 1:
            s += 1
            while not oracle.pad_check((ciphertext*modexp(s,e,n))%n): s += 1

        elif len(M) == 1:
            a, b = M[0]
            r = 2*(b*s - 2*B)//n
            s = (2*B + r*n)//b

            while not oracle.pad_check((ciphertext*modexp(s,e,n))%n): 
                s += 1
                if s > (3*B + r*n)//a:
                    r += 1
                    s = (2*B + r*n)//b

        M_next = []
        for (a,b) in M:
            r_min = (a*s - 3*B + n)//n
            r_max = (b*s - 2*B)//n

            for r in range(r_min,r_max+1):
                low = max(a,(2*B + r*n - 1)//s + 1)
                high = min(b,(3*B - 1 + r*n) // s)
                union(M_next,low,high)
        
        M = M_next
        i += 1
        if not i%1000: print('[=] updated {} rounds, M0-distance {}'.format(i,M[0][1]-M[0][0]))
        
    print('[+] converged at',M[0][0])    
    return int.to_bytes(M[0][0],key_length//8,'big')

def remove_pkcs1_padding(bytestring):
    m = re.compile(b'\x00\x02.{7,}\x00(.*)',re.DOTALL).search(bytestring)
    return m[1]

key_length = 256
data = b'kick it, CC'
oracle = RSA_Padding_Oracle(3,key_length//2)
padded_string = oracle.pad(data,key_length//8)
print('[=] padded data string {} to {}'.format(data,padded_string))
padded_string = int.from_bytes(padded_string,'big')
ciphertext = oracle.encrypt(padded_string)
print('[+] oracle verification of ciphertext:',oracle.pad_check(ciphertext))
m = attack_RSA_padding_oracle(oracle,ciphertext)
print('[+] plaintext found:',remove_pkcs1_padding(m))

[=] padded data string b'kick it, CC' to b'\x00\x02D\xa9\x9f\xd2\xe96\xe1\xea\x120\x82\xbd\x80\xab66\x14\xa0\x00kick it, CC'
[+] oracle verification of ciphertext: True
[=] attacking padding oracle
[+] initialisation for s done
[+] converged at 4007585943543401082003014823116966178101300411002878721222356020922172227
[+] plaintext found: b'kick it, CC'


### 48. Bleichenbacher's PKCS 1.5 Padding Oracle (Complete Case)

In [63]:
key_length = 768
data = b'kick it, CC'
oracle = RSA_Padding_Oracle(3,key_length//2)
padded_string = oracle.pad(data,key_length//8)
padded_string = int.from_bytes(padded_string,'big')
ciphertext = oracle.encrypt(padded_string)
m = attack_RSA_padding_oracle(oracle,ciphertext)
print('\nplaintext found:',remove_pkcs1_padding(m))

[=] attacking padding oracle
[=] tried 10000 values for s
[=] tried 20000 values for s
[+] initialisation for s done
[+] converged at 62404745254062008136230138769478829900431221747467682148466592952968205113705864343595460061384602102734430788745456094177973128798112158366695205726682942936413458061919810153401767524713383139334253623617610911138348594643779

plaintext found: b'kick it, CC'


### 49. CBC-MAC Message Forgery

In [64]:
key = key_gen(AES.block_size)

def CBC_MAC(plaintext,iv,key): return CBC_encrypt(plaintext,iv,key)[-AES.block_size:]

def make_transaction(to_acc,amount,iv=key_gen(AES.block_size)): 
    message = 'from=#me&to=#{}&amount=#{}'.format(to_acc,amount)
    mac = CBC_MAC(message,iv,key)
    return message.encode()+iv+mac
    
class Server:
    
    def __init__(self,key):
        self.key = key
        
    def take_message(self,query):
        if not self.verify(query)[0]: print('[FAILED] message validation failed'); return
        print('[+] validation ok')
        self.transaction(self.verify(query)[1].decode())
            
    def verify(self,query): 
        try:
            mac = query[-AES.block_size:]
            iv = query[-2*AES.block_size:-AES.block_size]
            message = query[:-2*AES.block_size]
            return CBC_MAC(message,iv,key) == mac,message
        except: return False,None
    
    def transaction(self,message):
        try:
            parsed = {pair.split('=#')[0]:pair.split("=#")[1] for pair in message.split('&')}
            print('[SUCCESS] Transferred {} from {} to {}'.format(parsed['amount'],parsed['from'],parsed['to']))
        except: print('[FAILED] transaction failed')

server = Server(key)

print("VERIFY CBC-MAC")
print("==================")
message = 'hello'
iv = key_gen(AES.block_size)
mac = CBC_MAC(message,iv,key)
print('message:',message)
print('mac:',mac)
print('server verification:',server.verify(message.encode()+iv+mac)[0])

print("\n\nSAMPLE TRANSACTION")
print("==================")
query = make_transaction('alice',225)
print('generated message for transaction:',query)
print('sending to server...')
server.take_message(query)

print("\n\nFORGE TRANSACTION WITH IV")
print("==================")
query = make_transaction('you','1M spacebucks',b'\x00'*AES.block_size)
print('[=] original message:',query)
message = query[:-2*AES.block_size]
mac = query[-AES.block_size:]
target_message = b'from=#you&to=#me&amount=#1M spacebucks'
iv = keyed_xor(message,target_message)
forged_query = target_message + iv[:AES.block_size] + mac
print('[=] forged message:',forged_query)
print('[=] sending to server...')
server.take_message(forged_query)

VERIFY CBC-MAC
message: hello
mac: b'm\x7f\xab\x80\xdce\xab\x91l\xa9\t\xc6n\xaf\x99\x9d'
server verification: True


SAMPLE TRANSACTION
generated message for transaction: b'from=#me&to=#alice&amount=#225\xfb\xc9\xce\x17\t\x10\x9b(\xa5*~\xf4^\x80r\x7ft\x130\x9a[\x85\xc3F\xabQ\x90s\x00\xab\x18\xdb'
sending to server...
[+] validation ok
[SUCCESS] Transferred 225 from me to alice


FORGE TRANSACTION WITH IV
[=] original message: b"from=#me&to=#you&amount=#1M spacebucks\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb0'\xcc)V\xc8\x81M\xa7\xa5\x1f9\xa5\xa5\xfeA"
[=] forged message: b"from=#you&to=#me&amount=#1M spacebucks\x00\x00\x00\x00\x00\x00\x14\nSR\x1bR\x1eZ\x02\x10\xb0'\xcc)V\xc8\x81M\xa7\xa5\x1f9\xa5\xa5\xfeA"
[=] sending to server...
[+] validation ok
[SUCCESS] Transferred 1M spacebucks from you to me


In [65]:
key = key_gen(AES.block_size)
iv = b'\x00'*AES.block_size

def make_transaction(to_accs,amounts): 
    tx = ';'.join([acc+':'+str(am) for acc,am in zip(to_accs,amounts)])
    message = 'from=#victim_account&tx_list=#' + tx
    mac = CBC_MAC(message,iv,key)
    return message.encode()+mac
    
class Efficient_Server:
    
    def __init__(self,key):
        self.key = key
        
    def take_message(self,query):
        if not self.verify(query)[0]: print('[-] message validation failed'); return
        print('[+] validation ok')
        self.transaction(self.verify(query)[1].decode())
            
    def verify(self,query): 
        try:
            mac = query[-AES.block_size:]
            message = query[:-AES.block_size]
            return CBC_MAC(message,iv,key) == mac,message
        except: return False,None
    
    def transaction(self,message):
        try:
            parsed = {pair.split('=#')[0]:pair.split("=#")[1] for pair in message.split('&')}
            for pair in parsed['tx_list'].split(';'): print('[SUCCESS] Transferred {} from {} to {}'.format(pair.split(':')[1],parsed['from'],pair.split(':')[0]))            
        except: print('[FAILED] transaction failed')
         
def limited_CBC_MAC(message): return CBC_MAC(message,iv,key)

server = Efficient_Server(key)

print("LEGITIMATE TRANSACTION")
print("==================")
query = make_transaction(['alice','bob','charlie'],[100,200,300])
print('generated message for transaction:',query)
print('sending to server...')
server.take_message(query)

print("\n\nLENGTH EXTENSION ATTACK")
print("==================")
mac = query[-AES.block_size:]
print('[=] original mac:',mac)
original_message = query[:-AES.block_size]
original_message = add_pad(original_message,AES.block_size)
print('[=] padded original message to align with block length:', original_message)
extension_stage_1 = b';attacker'
extension_stage_1 += b' '*(AES.block_size-len(extension_stage_1))
extension_stage_2 = b':1M spacebucks'
extension_stage_2 = add_pad(extension_stage_2,AES.block_size)
print('[=] extension stage 1:',extension_stage_1)
print('[=] extension stage 2:',extension_stage_2)
new_mac_stage_1 = limited_CBC_MAC(keyed_xor(keyed_xor(extension_stage_1,mac),iv))
new_mac_stage_2 = limited_CBC_MAC(keyed_xor(keyed_xor(new_mac_stage_1,extension_stage_2),iv))
forged_query = original_message + extension_stage_1 + b'\x10'*16 + extension_stage_2 + new_mac_stage_2
print('[=] forged message:',forged_query)
print('[=] sending to server...')
server.take_message(forged_query)


LEGITIMATE TRANSACTION
generated message for transaction: b'from=#victim_account&tx_list=#alice:100;bob:200;charlie:300\xa1\x1f\x0b[\xe2\xca\xa0$\xae\x8d=\xb0\xbd\xfb@\xf0'
sending to server...
[+] validation ok
[SUCCESS] Transferred 100 from victim_account to alice
[SUCCESS] Transferred 200 from victim_account to bob
[SUCCESS] Transferred 300 from victim_account to charlie


LENGTH EXTENSION ATTACK
[=] original mac: b'\xa1\x1f\x0b[\xe2\xca\xa0$\xae\x8d=\xb0\xbd\xfb@\xf0'
[=] padded original message to align with block length: b'from=#victim_account&tx_list=#alice:100;bob:200;charlie:300\x05\x05\x05\x05\x05'
[=] extension stage 1: b';attacker       '
[=] extension stage 2: b':1M spacebucks\x02\x02'
[=] forged message: b'from=#victim_account&tx_list=#alice:100;bob:200;charlie:300\x05\x05\x05\x05\x05;attacker       \x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10:1M spacebucks\x02\x02"\xed\xee\xee&\xe49<\xc1\xee\xe6d\x1b\xde\xe1+'
[=] sending to server...
[+] validation o

### 50. Hashing with CBC-MAC

In [66]:
def forge_CBC_MAC_hash(original_string,original_hash,new_string):
    new_string = add_pad(new_string,AES.block_size)
    first_stage_mac = CBC_MAC(new_string,iv,key)
    first_stage_padding = b'\x10'*16
    second_stage_first_block = keyed_xor(first_stage_mac,original_string[:AES.block_size])
    second_stage_rest = original_string[AES.block_size:]
    forged_string = new_string+first_stage_padding+second_stage_first_block+second_stage_rest
    return forged_string

key = b'YELLOW SUBMARINE'
iv = b'\x00'*16
original_string = b'alert(\'MZA who was that?\');\n'
print('[=] original string:',original_string)
target_hash = CBC_MAC(original_string,iv,key)
print('[=] target hash:',hexlify(target_hash).decode())

new_string = b'alert(\'Ayo, the Wu is back!\'); //'
forged_string = forge_CBC_MAC_hash(original_string,target_hash,new_string)
print('[+] forged string:',forged_string)
forged_hash = CBC_MAC(forged_string,iv,key)
print('[+] forged hash:',hexlify(forged_hash).decode())

[=] original string: b"alert('MZA who was that?');\n"
[=] target hash: 296b8d7cb78a243dda4d0a61d33bbdd1
[+] forged string: b"alert('Ayo, the Wu is back!'); //\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\xd4\x15\rY\x86\xe2*N\x91\xbc\x0f\xf7\xea_jqas that?');\n"
[+] forged hash: 296b8d7cb78a243dda4d0a61d33bbdd1


### 51. Compression Ratio Side-Channel Attacks

In [67]:
from zlib import compress

def generate_request(P):
    request = ""
    request += "POST / HTTP/1.1\n"
    request += "Host: hapless.com\n"
    request += "Cookie: sessionid=TmV2ZXIgcmV2ZWFsIHRoZSBXdS1UYW5nIFNlY3JldCE=\n"
    request += "Content-Length: " + str(len(P)) + "\n\n"
    request += P
    return request

def CBC_request_oracle(P): 
    key = key_gen(AES.block_size)
    iv = key_gen(AES.block_size)
    return len(CBC_encrypt(compress(generate_request(P).encode()),iv,key))

possible_chars = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890=\n'
payload = 'sessionid='
candidates = ['']
done = False
print('[+] attacking compression oracle')
for i in range(50):
    shortest = 99999
    head = ''
    while CBC_request_oracle(head+payload+candidates[0]) == CBC_request_oracle(payload+candidates[0]): head += 'A'
    for candidate in candidates:    
        for possible_char in possible_chars:
            test_payload = head + payload + candidate + possible_char
            test_length = CBC_request_oracle(test_payload)
            if test_length == shortest: winners.append(candidate + possible_char)
            if test_length < shortest: shortest,winners = test_length, [candidate + possible_char]
    candidates = winners
    
    for candidate in candidates: 
        if candidate[-2:] == '=\n': print('[+] got session id: ' + candidate); done = True
    if done: break
            
    if not i%5: print('[=] tested {} characters, {} candidates'.format(i+1,len(candidates)))

[+] attacking compression oracle
[=] tested 1 characters, 64 candidates
[=] tested 6 characters, 1 candidates
[=] tested 11 characters, 64 candidates
[=] tested 16 characters, 1 candidates
[=] tested 21 characters, 64 candidates
[=] tested 26 characters, 1 candidates
[=] tested 31 characters, 64 candidates
[=] tested 36 characters, 1 candidates
[=] tested 41 characters, 64 candidates
[+] got session id: TmV2ZXIgcmV2ZWFsIHRoZSBXdS1UYW5nIFNlY3JldCE=



### 52. Iterated Hash Function Multicollisions

In [68]:
def MD(message,hash,verbose=False):
    chunks = [add_pad(message[i:i+len(hash)],AES.block_size) for i in range(0,len(message),len(hash))]    
    all_hashes = []
    for chunk in chunks: 
        hash = AES_ECB_encrypt(chunk,add_pad(hash,AES.block_size))[:len(hash)]
        all_hashes.append(hash)
    return all_hashes if verbose else hash
    
def make_one_collision(limit,hash):
    found_hashes = []
    for m in range(256**limit):
        test_message = b''.join([bytes([(m%(256**(i+1)))//(256**i)]) for i in range(limit)])
        test_hash = MD(test_message,hash)
        if test_hash in found_hashes: 
            m1 = b''.join([bytes([(found_hashes.index(test_hash)%(256**(i+1)))//(256**i)]) for i in range(limit)])
            m2 = test_message
            collision_hash = test_hash
            print('[+] collision found: {} and {} both hash to {}'.format(m1,m2,collision_hash))
            return m1,m2,collision_hash
        else: found_hashes.append(test_hash)
            
            
def make_collision(n,limit,hash):
    collision_messages = [b'']
    for i in range(n):
        print('[=] looking for stage {} collision with initial hash {}'.format(i+1,hash))
        m1,m2,hash = make_one_collision(limit,hash)
        new_messages = []
        for collision_message in collision_messages: new_messages += [collision_message+m1,collision_message+m2]
        collision_messages = new_messages
    print('[+] done! found {} messages that collide'.format(len(collision_messages)))
    return collision_messages,hash
        
key = key_gen(AES.block_size)
limit = 3
n = 3
initial_hash = key_gen(limit)
collision_messages,hash = make_collision(n,limit,initial_hash)
print('[+] the following messages hash to {}:\n{}'.format(hash,collision_messages))
print('\nCHECK:')
for message in collision_messages: print('{} hashes to {}'.format(message,MD(message,initial_hash)))

[=] looking for stage 1 collision with initial hash b'\xf5]\x8f'
[+] collision found: b'\x01\x08\x00' and b'$\t\x00' both hash to b'/\xd7E'
[=] looking for stage 2 collision with initial hash b'/\xd7E'
[+] collision found: b'\xc7\x0c\x00' and b'\x0e\x12\x00' both hash to b'\x95;\x9a'
[=] looking for stage 3 collision with initial hash b'\x95;\x9a'
[+] collision found: b'\x18\t\x00' and b'\x93\x1f\x00' both hash to b'o\xdf\x15'
[+] done! found 8 messages that collide
[+] the following messages hash to b'o\xdf\x15':
[b'\x01\x08\x00\xc7\x0c\x00\x18\t\x00', b'\x01\x08\x00\xc7\x0c\x00\x93\x1f\x00', b'\x01\x08\x00\x0e\x12\x00\x18\t\x00', b'\x01\x08\x00\x0e\x12\x00\x93\x1f\x00', b'$\t\x00\xc7\x0c\x00\x18\t\x00', b'$\t\x00\xc7\x0c\x00\x93\x1f\x00', b'$\t\x00\x0e\x12\x00\x18\t\x00', b'$\t\x00\x0e\x12\x00\x93\x1f\x00']

CHECK:
b'\x01\x08\x00\xc7\x0c\x00\x18\t\x00' hashes to b'o\xdf\x15'
b'\x01\x08\x00\xc7\x0c\x00\x93\x1f\x00' hashes to b'o\xdf\x15'
b'\x01\x08\x00\x0e\x12\x00\x18\t\x00' hashes to

### 53. Kelsey and Schneier's Expandable Messages

In [69]:
def make_expandable(k,hash):
    
    pairs = []
    limit = len(hash)
    
    for i in range(k):
        length = 2**i+1
        print('[=] generating expandable pair of {} units'.format(length))
        short_hashes = []
        long_hashes = []
        long_hash_start = key_gen(length-1)*limit
        for m in range(256**limit):
            test_message = b''.join([bytes([(m%(256**(j+1)))//(256**j)]) for j in range(limit)])
            test_long_message = long_hash_start + test_message
            test_short_hash = MD(test_message,hash)
            test_long_hash = MD(test_long_message,hash)
            if test_short_hash in long_hashes:
                m2 = long_hash_start + b''.join([bytes([(long_hashes.index(test_short_hash)%(256**(j+1)))//(256**j)]) for j in range(limit)])
                m1 = test_message
                hash = test_short_hash
                break
            elif test_long_hash in short_hashes:
                m1 = b''.join([bytes([(short_hashes.index(test_long_hash)%(256**(j+1)))//(256**j)]) for j in range(limit)])
                m2 = test_long_message
                hash = test_long_hash
                break
            else:
                short_hashes.append(test_short_hash)
                long_hashes.append(test_long_hash)
        print('[+] collision found: {} and {} both hash to {}'.format(m1,m2,hash))
        pairs.append((m1,m2))
    return pairs,hash

def use_expandable(pairs,length,initial_hash):
    additional_dec = length - len(pairs)
    message = b''
    for i in range(len(pairs)): 
        bit = get_nth_bit(additional_dec,i+1,len(pairs))
        message += pairs[i][bit]
    return message

def find_bridge(intermediate_hash,map,k):
    print('[=] finding bridge')
    map = map[k:k+2**k-1]
    found_hashes = []
    limit = len(intermediate_hash)
    for m in range(256**limit):
        test_message = b''.join([bytes([(m%(256**(i+1)))//(256**i)]) for i in range(limit)])
        test_hash = MD(test_message,intermediate_hash)
        if test_hash in map: 
            m1 = test_message
            index = map.index(test_hash) + k
            print('[+] found bridge {} to point after {} blocks'.format(m1,index))
            return m1,index
        else: found_hashes.append(test_hash)
    print('[-] couldn\'t find bridge point')


limit = 3
k = 3
initial_hash = key_gen(limit)
target_message = b'purple train to groovy city!'

print("GENERATE EXPANDABLE MESSAGES")
print("==================")
pairs,hash = make_expandable(k,initial_hash)
print('[+] done! can generate collision messages between {} and {} blocks'.format(k,k+2**k-1))
print('\nCHECK:')
for i in range(k,k+2**k): 
    message = use_expandable(pairs,i,initial_hash)
    print('length {} message {} hashes to {}'.format(i,message,MD(message,initial_hash)))

print("\n\nFORGE MESSAGE")
print("==================")
map = MD(target_message,initial_hash,True)
print('[=] target hash: {}'.format(map[-1]))
bridge,point = find_bridge(hash,map,k)
forged_message = use_expandable(pairs,point,initial_hash) + bridge + target_message[(point+1)*limit:]
print('[=] forged message: {}'.format(forged_message))
print('[+] forged hash: {}'.format(MD(forged_message,initial_hash)))

GENERATE EXPANDABLE MESSAGES
[=] generating expandable pair of 2 units
[+] collision found: b'\xf1\x04\x00' and b'BBBK\x08\x00' both hash to b'\xd6&\x8f'
[=] generating expandable pair of 3 units
[+] collision found: b'\xf5\x03\x00' and b'\xeb3\xeb3\xeb3[\x1d\x00' both hash to b'\xa5\x17J'
[=] generating expandable pair of 5 units
[+] collision found: b'\xac\x00\x00' and b'3\xdc.\xa33\xdc.\xa33\xdc.\xa3a\t\x00' both hash to b's\r1'
[+] done! can generate collision messages between 3 and 10 blocks

CHECK:
length 3 message b'\xf1\x04\x00\xf5\x03\x00\xac\x00\x00' hashes to b's\r1'
length 4 message b'BBBK\x08\x00\xf5\x03\x00\xac\x00\x00' hashes to b's\r1'
length 5 message b'\xf1\x04\x00\xeb3\xeb3\xeb3[\x1d\x00\xac\x00\x00' hashes to b's\r1'
length 6 message b'BBBK\x08\x00\xeb3\xeb3\xeb3[\x1d\x00\xac\x00\x00' hashes to b's\r1'
length 7 message b'\xf1\x04\x00\xf5\x03\x003\xdc.\xa33\xdc.\xa33\xdc.\xa3a\t\x00' hashes to b's\r1'
length 8 message b'BBBK\x08\x00\xf5\x03\x003\xdc.\xa33\xdc.\xa33\x

### 54. Kelsey and Kohno's Nostradamus Attack

In [70]:
def collide_states(s1,s2):
    hashes1 = []
    hashes2 = []
    limit = len(s1)
    for m in range(256**limit):
        test_message = b''.join([bytes([(m%(256**(j+1)))//(256**j)]) for j in range(limit)])
        hash1 = MD(test_message,s1)
        hash2 = MD(test_message,s2)
        
        if hash1 in hashes2:
            m1 = test_message
            m2 = b''.join([bytes([(hashes2.index(hash1)%(256**(j+1)))//(256**j)]) for j in range(limit)])
            return m1,m2,hash1
        
        hashes1.append(hash1)
        
        if hash2 in hashes1:
            m1 = b''.join([bytes([(hashes1.index(hash2)%(256**(j+1)))//(256**j)]) for j in range(limit)])
            m2 = test_message
            return m1,m2,hash2
        
        hashes2.append(hash2)
    
def funnel(possible_start_states):
    print('[=] funnelling possible start states')
    messages = [b'']*len(possible_start_states)
    reps = 1
    while reps < len(possible_start_states):
        for i in range(0,len(possible_start_states),reps*2): 
            if reps == 1: m1,m2,hash = collide_states(possible_start_states[i],possible_start_states[i+reps])
            else: m1,m2,hash = collide_states(MD(messages[i],possible_start_states[i]),MD(messages[i+reps],possible_start_states[i+reps]))
            for j in range(reps): 
                messages[i+j] += m1
                messages[i+reps+j] += m2
        reps *= 2
        if reps < len(possible_start_states): print('[+] funnelled to {} states'.format(len(possible_start_states)//reps))
    prediction = MD(messages[0],possible_start_states[0])
    print('[+] done! all messages hash to {}'.format(prediction))
    return messages,prediction
            
k = 3
limit = 3
initial_hash = key_gen(limit)
possible_results = ['result'+str(i) for i in range(2**k)]
possible_start_states = [MD(add_pad(result,limit),initial_hash) for result in possible_results]

print("FUNNEL STATES")
print("==================")
print('[=] start states: {}'.format(possible_start_states))
messages,prediction = funnel(possible_start_states)

print('\nCHECK:')
for state,message in zip(possible_start_states,messages): print('hash of {} from {} is {}'.format(message,state,MD(message,state)))
    
print("\n\nFORGE PREDICTION")
print("==================")
print('[=] prediction:',prediction)
result = random.choice(possible_results)
print('[=] result:',result)
real_start_state = MD(add_pad(result,limit),initial_hash)
message = add_pad(result,limit) + messages[possible_start_states.index(real_start_state)]
print('[=] message:',message)
print('[+] hash of message:',MD(message,initial_hash))

FUNNEL STATES
[=] start states: [b'--\xbb', b'MS\x9a', b'.\x01*', b'\x10\x82\xf1', b'=$\xfb', b'\xd2L9', b'\x08\x03\x9b', b'\xbe\x07\xab']
[=] funnelling possible start states
[+] funnelled to 4 states
[+] funnelled to 2 states
[+] done! all messages hash to b'*\x92\xec'

CHECK:
hash of b'\x00\x05\x00S\x04\x00\xbf\x08\x00' from b'--\xbb' is b'*\x92\xec'
hash of b'\xbd\x06\x00S\x04\x00\xbf\x08\x00' from b'MS\x9a' is b'*\x92\xec'
hash of b'\x1e\x01\x00\x9b\x01\x00\xbf\x08\x00' from b'.\x01*' is b'*\x92\xec'
hash of b'\x99\x00\x00\x9b\x01\x00\xbf\x08\x00' from b'\x10\x82\xf1' is b'*\x92\xec'
hash of b'\x94\x02\x00\xff\x0c\x00\xf2\x04\x00' from b'=$\xfb' is b'*\x92\xec'
hash of b'\xce\x06\x00\xff\x0c\x00\xf2\x04\x00' from b'\xd2L9' is b'*\x92\xec'
hash of b'\xb6\x06\x00d\x05\x00\xf2\x04\x00' from b'\x08\x03\x9b' is b'*\x92\xec'
hash of b'I\r\x00d\x05\x00\xf2\x04\x00' from b'\xbe\x07\xab' is b'*\x92\xec'


FORGE PREDICTION
[=] prediction: b'*\x92\xec'
[=] result: result7
[=] message: b'resu